#Importar bibliotecas

In [ ]:
import numpy                  as np
import pandas                 as pd
import nltk
import re
from nltk.corpus import stopwords
from math import nan, isnan
from nltk.tokenize import word_tokenize
import unicodedata
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.util import ngrams
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import datetime
from datetime import datetime

pd.set_option('precision', 0)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Analisando tabelas

## Lista de deputados

In [ ]:
lista_deputados = pd.read_csv('/content/drive/MyDrive/Radar Virtù/Radar Congresso/Banco de dados brutos/deputados.csv', sep=';')

In [ ]:
lista = lista_deputados[lista_deputados.idLegislaturaFinal == 56]
lista

,uri,nome,idLegislaturaInicial,idLegislaturaFinal,nomeCivil,cpf,siglaSexo,urlRedeSocial,urlWebsite,dataNascimento,dataFalecimento,ufNascimento,municipioNascimento
0,https://dadosabertos.camara.leg.br/api/v2/depu...,Abílio Santana,56,56,JOSE ABILIO SILVA DE SANTANA,NaN,M,NaN,NaN,1965-02-13,NaN,BA,Salvador
1,https://dadosabertos.camara.leg.br/api/v2/depu...,Abou Anni,56,56,PAULO SERGIO ABOU ANNI,NaN,M,https://www.facebook.com/DeputadoFederalAbouAn...,http://www.abouanni.com.br,1966-11-06,NaN,SP,São Paulo
2,https://dadosabertos.camara.leg.br/api/v2/depu...,Acácio Favacho,56,56,ACÁCIO DA SILVA FAVACHO NETO,NaN,M,"https://twitter.com/acaciofavacho, https://www...",NaN,1983-09-28,NaN,AP,Macapá
3,https://dadosabertos.camara.leg.br/api/v2/depu...,Adolfo Viana,56,56,ADOLFO VIANA DE CASTRO NETO,NaN,M,NaN,NaN,1981-02-02,NaN,BA,Salvador
4,https://dadosabertos.camara.leg.br/api/v2/depu...,Adriana Ventura,56,56,ADRIANA MIGUEL VENTURA,NaN,F,"https://twitter.com/adriventurasp, https://www...",NaN,1969-03-06,NaN,SP,São Paulo
...,...,...,...,...,...,...,...,...,...,...,...,...,...
593,https://dadosabertos.camara.leg.br/api/v2/depu...,Zé Carlos,55,56,JOSÉ CARLOS NUNES JÚNIOR,NaN,M,NaN,NaN,1955-05-26,NaN,MA,São Luís
594,https://dadosabertos.camara.leg.br/api/v2/depu...,Zé Neto,56,56,JOSÉ CERQUEIRA DE SANTANA NETO,NaN,M,NaN,NaN,1964-03-30,NaN,BA,Feira de Santana
595,https://dadosabertos.camara.leg.br/api/v2/depu...,Zé Silva,54,56,JOSÉ SILVA SOARES,NaN,M,NaN,NaN,1963-05-11,NaN,MG,Iturama
596,https://dadosabertos.camara.leg.br/api/v2/depu...,Zé Vitor,56,56,JOSE VITOR DE RESENDE AGUIAR,NaN,M,"https://www.facebook.com/zevitorMG, https://ww...",NaN,1984-11-01,NaN,MG,Araguari


In [ ]:
lista_dos_deputados = lista[['nome']]
lista_dos_deputados

,nome
0,Abílio Santana
1,Abou Anni
2,Acácio Favacho
3,Adolfo Viana
4,Adriana Ventura
...,...
593,Zé Carlos
594,Zé Neto
595,Zé Silva
596,Zé Vitor


## Autores das proposições

In [ ]:
proposicoes_autores2021_2022 = pd.read_csv('/content/drive/MyDrive/Radar Virtù/Radar Congresso/Banco de dados brutos/proposicoes_autores.csv')
proposicoes_autores2019 = pd.read_csv('/content/drive/MyDrive/Radar Virtù/Radar Congresso/Banco de dados brutos/proposicoesAutores-2019.csv', sep=';')
proposicoes_autores2020 = pd.read_csv('/content/drive/MyDrive/Radar Virtù/Radar Congresso/Banco de dados brutos/proposicoesAutores-2020.csv', sep=';')
proposicoes_autores = pd.concat([proposicoes_autores2019, proposicoes_autores2020, proposicoes_autores2021_2022])
proposicoes_autores

,idProposicao,uriProposicao,idDeputadoAutor,uriAutor,codTipoAutor,tipoAutor,nomeAutor,siglaPartidoAutor,uriPartidoAutor,siglaUFAutor,ordemAssinatura,proponente,Ano
0,2344907,https://dadosabertos.camara.leg.br/api/v2/prop...,NaN,https://dadosabertos.camara.leg.br/api/v2/orga...,40000,Órgão do Poder Legislativo,Senado Federal - Alvaro Dias,PODEMOS,NaN,PR,1,1,NaN
1,2344525,https://dadosabertos.camara.leg.br/api/v2/prop...,NaN,https://dadosabertos.camara.leg.br/api/v2/orga...,40000,Órgão do Poder Legislativo,Senado Federal - Flávio Arns,PODEMOS,NaN,PR,1,1,NaN
2,2344523,https://dadosabertos.camara.leg.br/api/v2/prop...,NaN,https://dadosabertos.camara.leg.br/api/v2/orga...,40000,Órgão do Poder Legislativo,Senado Federal - Plínio Valério,PSDB,NaN,AM,1,1,NaN
3,2344522,https://dadosabertos.camara.leg.br/api/v2/prop...,NaN,https://dadosabertos.camara.leg.br/api/v2/orga...,40000,Órgão do Poder Legislativo,Senado Federal- Eduardo Girão,PODEMOS,NaN,CE,1,1,NaN
4,2343468,https://dadosabertos.camara.leg.br/api/v2/prop...,204457,https://dadosabertos.camara.leg.br/api/v2/depu...,10000,Deputado,Juninho do Pneu,UNIÃO,https://dadosabertos.camara.leg.br/api/v2/part...,RJ,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117083,1301128,https://dadosabertos.camara.leg.br/api/v2/prop...,178862,https://dadosabertos.camara.leg.br/api/v2/depu...,10000,Deputado,Tia Eron,PRB,https://dadosabertos.camara.leg.br/api/v2/part...,BA,1,1,2021
117084,946514,https://dadosabertos.camara.leg.br/api/v2/prop...,178931,https://dadosabertos.camara.leg.br/api/v2/depu...,10000,Deputado,Luciano Ducci,PSB,https://dadosabertos.camara.leg.br/api/v2/part...,PR,1,1,2021
117085,540979,https://dadosabertos.camara.leg.br/api/v2/prop...,160666,https://dadosabertos.camara.leg.br/api/v2/depu...,10000,Deputado,Félix Mendonça Júnior,PDT,https://dadosabertos.camara.leg.br/api/v2/part...,BA,1,1,2021
117086,498347,https://dadosabertos.camara.leg.br/api/v2/prop...,160649,https://dadosabertos.camara.leg.br/api/v2/depu...,10000,Deputado,Esperidião Amin,PP,https://dadosabertos.camara.leg.br/api/v2/part...,SC,1,1,2021


In [ ]:
proposicoes_autores['tipoAutor'].drop_duplicates()

0               Órgão do Poder Legislativo
4                                 Deputado
5                         COMISSÃO EXTERNA
51                     COMISSÃO PERMANENTE
155               Órgão do Poder Executivo
157                                Senador
269                        Sociedade Civil
519                            SUBCOMISSÃO
541              COMISSÃO MISTA PERMANENTE
713      MPU - Ministério Público da União
868                              Liderança
1098                     COMISSÃO ESPECIAL
1412               Órgão do Senado Federal
1474            COMISSÃO MEDIDA PROVISÓRIA
2437                     COMISSÃO DIRETORA
2898                      Partido Político
5322                              CONSELHO
6137             Órgão do Poder Judiciário
30972    COMISSÃO PARLAMENTAR DE INQUÉRITO
51300        Órgão da Câmara dos Deputados
312                      GRUPO DE TRABALHO
1814     DPU - Defensoria Pública da União
82977         PERMANENTE DO SENADO FEDERAL
Name: tipoA

In [ ]:
selecao_politicos = ['Deputado', 'Senador']

In [ ]:
selecao_deputados = proposicoes_autores.loc[proposicoes_autores['tipoAutor'].isin(selecao_politicos)]
selecao_deputados

,idProposicao,uriProposicao,idDeputadoAutor,uriAutor,codTipoAutor,tipoAutor,nomeAutor,siglaPartidoAutor,uriPartidoAutor,siglaUFAutor,ordemAssinatura,proponente,Ano
4,2343468,https://dadosabertos.camara.leg.br/api/v2/prop...,204457,https://dadosabertos.camara.leg.br/api/v2/depu...,10000,Deputado,Juninho do Pneu,UNIÃO,https://dadosabertos.camara.leg.br/api/v2/part...,RJ,1,1,NaN
7,2338915,https://dadosabertos.camara.leg.br/api/v2/prop...,204572,https://dadosabertos.camara.leg.br/api/v2/depu...,10000,Deputado,Capitão Alberto Neto,PL,https://dadosabertos.camara.leg.br/api/v2/part...,AM,1,1,NaN
8,2338667,https://dadosabertos.camara.leg.br/api/v2/prop...,178916,https://dadosabertos.camara.leg.br/api/v2/depu...,10000,Deputado,Daniel Coelho,CIDADANIA,https://dadosabertos.camara.leg.br/api/v2/part...,PE,1,1,NaN
14,2330989,https://dadosabertos.camara.leg.br/api/v2/prop...,141398,https://dadosabertos.camara.leg.br/api/v2/depu...,10000,Deputado,Carlos Zarattini,PT,https://dadosabertos.camara.leg.br/api/v2/part...,SP,1,1,NaN
84,2297229,https://dadosabertos.camara.leg.br/api/v2/prop...,160532,https://dadosabertos.camara.leg.br/api/v2/depu...,10000,Deputado,Eli Corrêa Filho,DEM,https://dadosabertos.camara.leg.br/api/v2/part...,SP,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
117083,1301128,https://dadosabertos.camara.leg.br/api/v2/prop...,178862,https://dadosabertos.camara.leg.br/api/v2/depu...,10000,Deputado,Tia Eron,PRB,https://dadosabertos.camara.leg.br/api/v2/part...,BA,1,1,2021
117084,946514,https://dadosabertos.camara.leg.br/api/v2/prop...,178931,https://dadosabertos.camara.leg.br/api/v2/depu...,10000,Deputado,Luciano Ducci,PSB,https://dadosabertos.camara.leg.br/api/v2/part...,PR,1,1,2021
117085,540979,https://dadosabertos.camara.leg.br/api/v2/prop...,160666,https://dadosabertos.camara.leg.br/api/v2/depu...,10000,Deputado,Félix Mendonça Júnior,PDT,https://dadosabertos.camara.leg.br/api/v2/part...,BA,1,1,2021
117086,498347,https://dadosabertos.camara.leg.br/api/v2/prop...,160649,https://dadosabertos.camara.leg.br/api/v2/depu...,10000,Deputado,Esperidião Amin,PP,https://dadosabertos.camara.leg.br/api/v2/part...,SC,1,1,2021


In [ ]:
proposicoes_autores1 = selecao_deputados[['idProposicao', 'idDeputadoAutor', 'nomeAutor', 'siglaPartidoAutor', 'siglaUFAutor']]
proposicoes_autores1

,idProposicao,idDeputadoAutor,nomeAutor,siglaPartidoAutor,siglaUFAutor
4,2343468,204457,Juninho do Pneu,UNIÃO,RJ
7,2338915,204572,Capitão Alberto Neto,PL,AM
8,2338667,178916,Daniel Coelho,CIDADANIA,PE
14,2330989,141398,Carlos Zarattini,PT,SP
84,2297229,160532,Eli Corrêa Filho,DEM,SP
...,...,...,...,...,...
117083,1301128,178862,Tia Eron,PRB,BA
117084,946514,178931,Luciano Ducci,PSB,PR
117085,540979,160666,Félix Mendonça Júnior,PDT,BA
117086,498347,160649,Esperidião Amin,PP,SC


In [ ]:
proposicoes_autores1.columns = ['id_proposicao', 'id_deputado', 'politico', 'partido', 'uf']
proposicoes_autores1.head()

,id_proposicao,id_deputado,politico,partido,uf
4,2343468,204457,Juninho do Pneu,UNIÃO,RJ
7,2338915,204572,Capitão Alberto Neto,PL,AM
8,2338667,178916,Daniel Coelho,CIDADANIA,PE
14,2330989,141398,Carlos Zarattini,PT,SP
84,2297229,160532,Eli Corrêa Filho,DEM,SP


In [ ]:
proposicoes_autores1['politico'].drop_duplicates()

4              Juninho do Pneu
7         Capitão Alberto Neto
8                Daniel Coelho
14            Carlos Zarattini
84            Eli Corrêa Filho
                  ...         
98018            Norma Pereira
103090            Jorge Bittar
113815          Rachel Marques
117082                  Bebeto
117087            Marçal Filho
Name: politico, Length: 767, dtype: object

## Lista das proposições

In [ ]:
proposicoes_2019 = pd.read_csv('/content/drive/MyDrive/Radar Virtù/Radar Congresso/Banco de dados brutos/proposicoes-2019.csv', sep=';')
proposicoes_2020 = pd.read_csv('/content/drive/MyDrive/Radar Virtù/Radar Congresso/Banco de dados brutos/proposicoes-2020.csv', sep=';')
proposicoes_2021 = pd.read_csv('/content/drive/MyDrive/Radar Virtù/Radar Congresso/Banco de dados brutos/proposicoes-2021.csv', sep=';')
proposicoes_2022 = pd.read_csv('/content/drive/MyDrive/Radar Virtù/Radar Congresso/Banco de dados brutos/proposicoes-2022.csv', sep=';')
proposicoes_2023 = pd.read_csv('/content/drive/MyDrive/Radar Virtù/Radar Congresso/Banco de dados brutos/proposicoes-2023.csv', sep=';')

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
proposicoes_2019['ano'] = '2019'
proposicoes_2020['ano'] = '2020'
proposicoes_2021['ano'] = '2021'
proposicoes_2022['ano'] = '2022'
proposicoes_2023['ano'] = '2023'

In [ ]:
proposicoes1 = pd.concat([proposicoes_2019, proposicoes_2020, proposicoes_2021, proposicoes_2022])
proposicoes1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132549 entries, 0 to 24845
Data columns (total 31 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   id                                132549 non-null  int64  
 1   uri                               132549 non-null  object 
 2   siglaTipo                         132549 non-null  object 
 3   numero                            132549 non-null  int64  
 4   ano                               132549 non-null  object 
 5   codTipo                           132549 non-null  int64  
 6   descricaoTipo                     132549 non-null  object 
 7   ementa                            127855 non-null  object 
 8   ementaDetalhada                   1253 non-null    object 
 9   keywords                          20718 non-null   object 
 10  dataApresentacao                  132549 non-null  object 
 11  uriOrgaoNumerador                 132549 non-null  ob

In [ ]:
proposicoes1.head(3)

,id,uri,siglaTipo,numero,ano,codTipo,descricaoTipo,ementa,ementaDetalhada,keywords,...,ultimoStatus_siglaOrgao,ultimoStatus_uriOrgao,ultimoStatus_regime,ultimoStatus_descricaoTramitacao,ultimoStatus_idTipoTramitacao,ultimoStatus_descricaoSituacao,ultimoStatus_idSituacao,ultimoStatus_despacho,ultimoStatus_apreciacao,ultimoStatus_url
0,308880,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,2441,2019,139,Projeto de Lei,Dá nova redação aos arts. 20 e 123 do Decreto-...,"Exige dos corretores de seguro e resseguro, pe...","Alteração, Lei do Seguro Privado, obrigatoried...",...,CFT,https://dadosabertos.camara.leg.br/api/v2/orga...,"Prioridade (Art. 151, II, RICD)",Devolução ao Relator,170,Aguardando Parecer,915,"Devolvido ao Relator, Dep. Lucas Vergilio (SOL...",Proposição Sujeita à Apreciação do Plenário,NaN
1,317970,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,2037,2019,139,Projeto de Lei,"Altera a Lei nº 8.666, de 21 de junho de 1993,...",NaN,"Alteração, Lei das Licitações, proibição, ante...",...,CTASP,https://dadosabertos.camara.leg.br/api/v2/orga...,"Ordinário (Art. 151, III, RICD)",Publicação de Proposição,604,Aguardando Designação de Relator,907,Encaminhada à publicação. Publicação Inicial e...,Proposição Sujeita à Apreciação do Plenário,http://www.camara.gov.br/proposicoesWeb/prop_m...
2,427339,https://dadosabertos.camara.leg.br/api/v2/prop...,PL,2439,2019,139,Projeto de Lei,"Altera o art. 125 da Lei nº 9.279, de 14 de ma...",NaN,"Alteração, Nova Lei de Patentes, critérios, re...",...,CCJC,https://dadosabertos.camara.leg.br/api/v2/orga...,"Ordinário (Art. 151, III, RICD)",Saída de Relator da Comissão - Com Parecer Apr...,1255,Aguardando Designação de Relator,907,"O Relator, Dep. Luiz Philippe de Orleans e Bra...",Proposição Sujeita à Apreciação do Plenário,NaN


In [ ]:
proposicoes1['ano'].value_counts()

2021    39749
2019    38264
2020    29690
2022    24846
Name: ano, dtype: int64

In [ ]:
proposicoes1['data'] = pd.to_datetime(proposicoes1['ultimoStatus_dataHora']).dt.date
proposicoes1['hora'] = pd.to_datetime(proposicoes1['ultimoStatus_dataHora']).dt.time

In [ ]:
proposicoes_selecao = proposicoes1[['ano', 'id', 'descricaoTipo', 'ementa', 'keywords', 'ultimoStatus_despacho', 'data']]
proposicoes_selecao.columns = ['ano', 'id_proposicao', 'tipo', 'ementa', 'palavras_chave', 'status', 'data']
proposicoes_selecao.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132549 entries, 0 to 24845
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   ano             132549 non-null  object
 1   id_proposicao   132549 non-null  int64 
 2   tipo            132549 non-null  object
 3   ementa          127855 non-null  object
 4   palavras_chave  20718 non-null   object
 5   status          132541 non-null  object
 6   data            132549 non-null  object
dtypes: int64(1), object(6)
memory usage: 8.1+ MB


In [ ]:
proposicoes_selecao.head(2)

,ano,id_proposicao,tipo,ementa,palavras_chave,status,data
0,2019,308880,Projeto de Lei,Dá nova redação aos arts. 20 e 123 do Decreto-...,"Alteração, Lei do Seguro Privado, obrigatoried...","Devolvido ao Relator, Dep. Lucas Vergilio (SOL...",2019-06-26
1,2019,317970,Projeto de Lei,"Altera a Lei nº 8.666, de 21 de junho de 1993,...","Alteração, Lei das Licitações, proibição, ante...",Encaminhada à publicação. Publicação Inicial e...,2019-04-16


In [ ]:
proposicoes_selecao[proposicoes_selecao['ementa'].isnull()]

,ano,id_proposicao,tipo,ementa,palavras_chave,status,data
19,2019,2080812,Emenda de Plenário,NaN,NaN,Apresentação da Emenda de Plenário n. 1/2019 P...,2019-03-12
46,2019,2190336,Emenda de Relator,NaN,NaN,"Apresentação da Emenda de Relator n. 1 CFT, pe...",2019-01-08
1160,2019,2191672,Emenda de Plenário,NaN,NaN,Apresentação da Emenda de Plenário n. 5/2019 P...,2019-02-12
1175,2019,2191688,Emenda de Plenário,NaN,NaN,Apresentação da Emenda de Plenário n. 6/2019 P...,2019-02-12
1176,2019,2191689,Emenda de Plenário,NaN,NaN,Apresentação da Emenda de Plenário n. 7/2019 P...,2019-02-12
...,...,...,...,...,...,...,...
24803,2022,2342294,Destaque,NaN,NaN,"Apresentação do Destaque n. 11 PLEN, pelo Depu...",2022-12-13
24818,2022,2342311,Complementação de Voto,NaN,NaN,Apresentação da Complementação de Voto n. 1 CS...,2022-12-13
24833,2022,2342327,Requerimento Procedimental de Sessão/Reunião,NaN,NaN,Relatório de Conferência de Assinaturas Eletrô...,2022-12-13
24837,2022,2342331,Subemenda Substitutiva de Plenário,NaN,NaN,Apresentação da Subemenda Substitutiva de Plen...,2022-12-13


In [ ]:
proposicoes_selecao.dropna(subset = ['ementa'], inplace = True)
proposicoes_selecao.index = range(proposicoes_selecao.shape[0])
proposicoes_selecao.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 127855 entries, 0 to 127854
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   ano             127855 non-null  object
 1   id_proposicao   127855 non-null  int64 
 2   tipo            127855 non-null  object
 3   ementa          127855 non-null  object
 4   palavras_chave  20712 non-null   object
 5   status          127847 non-null  object
 6   data            127855 non-null  object
dtypes: int64(1), object(6)
memory usage: 6.8+ MB


/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
proposicoes_selecao

,ano,id_proposicao,tipo,ementa,palavras_chave,status,data
0,2019,308880,Projeto de Lei,Dá nova redação aos arts. 20 e 123 do Decreto-...,"Alteração, Lei do Seguro Privado, obrigatoried...","Devolvido ao Relator, Dep. Lucas Vergilio (SOL...",2019-06-26
1,2019,317970,Projeto de Lei,"Altera a Lei nº 8.666, de 21 de junho de 1993,...","Alteração, Lei das Licitações, proibição, ante...",Encaminhada à publicação. Publicação Inicial e...,2019-04-16
2,2019,427339,Projeto de Lei,"Altera o art. 125 da Lei nº 9.279, de 14 de ma...","Alteração, Nova Lei de Patentes, critérios, re...","O Relator, Dep. Luiz Philippe de Orleans e Bra...",2022-04-27
3,2019,501638,Projeto de Lei,Institui o Programa Cidade Amiga do Idoso.,"Criação, Programa Cidade Amiga do Idoso, quali...",Encaminhada à publicação. Parecer da Comissão ...,2021-08-02
4,2019,512431,Projeto de Lei,Proíbe a utilização de mensagens subliminares ...,"Alteração, Código Brasileiro de Telecomunicaçõ...",Encaminhada à publicação. Parecer da Comissão ...,2021-10-01
...,...,...,...,...,...,...,...
127850,2022,2342334,Destaque de Emenda,Destaque de Emenda - PL 4363/2001,NaN,"Apresentação do Destaque de Emenda n. 13 PLEN,...",2022-12-13
127851,2022,2342335,Parecer Preliminar às Emendas de Plenário,Institui o Estatuto Nacional de Simplificação ...,NaN,Apresentação do Parecer Preliminar às Emendas ...,2022-12-13
127852,2022,2342337,Requerimento de Urgência (Art. 155 do RICD),"Com base no art. 155 do Regimento Interno, req...",NaN,Relatório de Conferência de Assinaturas Eletrô...,2022-12-13
127853,2022,2342338,Requerimento de Urgência (Art. 155 do RICD),"Com base no art. 155 do Regimento Interno, req...",NaN,Relatório de Conferência de Assinaturas Eletrô...,2022-12-13


## Juntar bases de dados

In [ ]:
proposicoes2 = pd.merge(proposicoes_autores1, proposicoes_selecao, on='id_proposicao')
proposicoes2

,id_proposicao,id_deputado,politico,partido,uf,ano,tipo,ementa,palavras_chave,status,data
0,2343468,204457,Juninho do Pneu,UNIÃO,RJ,2019,Parecer do Relator,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",NaN,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",2019-07-03
1,2330989,141398,Carlos Zarattini,PT,SP,2019,Voto em Separado,"Parecer do Relator, Dep. Carlos Zarattini (PT-...",NaN,Transformação do Parecer do Relator no Voto em...,2022-06-08
2,2297229,160532,Eli Corrêa Filho,DEM,SP,2019,Emenda ao Substitutivo,"Acrescenta o artigo 43-A a Lei nº 8.078, de 11...",NaN,Apresentação da Emenda ao Substitutivo n. 1 CD...,2019-05-14
3,2293392,160556,Padre João,PT,MG,2019,Substitutivo,"APROVA o Projeto de Lei nº 932, de 2015, que a...",NaN,"Apresentação do Substitutivo n. 2 CDHM, pelo D...",2019-05-22
4,2293391,160556,Padre João,PT,MG,2019,Parecer do Relator,"Parecer do Relator, Dep. Padre João (PT-MG), p...",NaN,"Parecer do Relator, Dep. Padre João (PT-MG), p...",2019-05-22
...,...,...,...,...,...,...,...,...,...,...,...
278429,1301128,178862,Tia Eron,PRB,BA,2021,Projeto de Lei,Tipifica o crime de injúria racial coletiva e ...,"Alteração, Código Penal, Lei Antirracismo, cri...",A matéria vai à sanção (PL 4.566-C/2021).,2022-12-07
278430,946514,178931,Luciano Ducci,PSB,PR,2021,Projeto de Lei,"Institui a ""Semana Nacional do Uso Consciente ...","Criação, Semana Nacional do Uso Consciente da ...",Encaminhada à publicação. Parecer da Comissão ...,2021-12-09
278431,540979,160666,Félix Mendonça Júnior,PDT,BA,2021,Projeto de Lei,Cria o Selo Verde Cacau Cabruca.,"Criação, Selo Verde Cacau Cabruca, sustentabil...","Designado Relator, Dep. Evair Vieira de Melo (...",2022-06-29
278432,498347,160649,Esperidião Amin,PP,SC,2021,Projeto de Lei,"Altera a Lei nº 9.277, de 10 de maio de 1996, ...","Alteração, isenção, pedágio, residente, exerc...",Transformado na Lei Ordinária 14157/2021. DOU ...,2021-06-01


In [ ]:
proposicoes2['politico'] = proposicoes2['politico'].str.lower().map(lambda x: re.sub(r'[^\w\s]', '', x))
proposicoes2.head()

,id_proposicao,id_deputado,politico,partido,uf,ano,tipo,ementa,palavras_chave,status,data
0,2343468,204457,juninho do pneu,UNIÃO,RJ,2019,Parecer do Relator,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",NaN,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",2019-07-03
1,2330989,141398,carlos zarattini,PT,SP,2019,Voto em Separado,"Parecer do Relator, Dep. Carlos Zarattini (PT-...",NaN,Transformação do Parecer do Relator no Voto em...,2022-06-08
2,2297229,160532,eli corrêa filho,DEM,SP,2019,Emenda ao Substitutivo,"Acrescenta o artigo 43-A a Lei nº 8.078, de 11...",NaN,Apresentação da Emenda ao Substitutivo n. 1 CD...,2019-05-14
3,2293392,160556,padre joão,PT,MG,2019,Substitutivo,"APROVA o Projeto de Lei nº 932, de 2015, que a...",NaN,"Apresentação do Substitutivo n. 2 CDHM, pelo D...",2019-05-22
4,2293391,160556,padre joão,PT,MG,2019,Parecer do Relator,"Parecer do Relator, Dep. Padre João (PT-MG), p...",NaN,"Parecer do Relator, Dep. Padre João (PT-MG), p...",2019-05-22


In [ ]:
siglas_df = pd.DataFrame([['Acre', 'AC', 'Norte'], ['Alagoas', 'AL', 'Nordeste'], ['Amapá', 'AP', 'Norte'], ['Amazonas', 'AM', 'Norte'], 
                          ['Bahia', 'BA', 'Nordeste'], ['Ceará', 'CE', 'Nordeste'], ['Distrito Federal', 'DF', 'Centro-Oeste'], 
                          ['Espírito Santo', 'ES', 'Sudeste'], ['Goiás', 'GO', 'Centro-Oeste'], ['Maranhão', 'MA', 'Nordeste'], 
                          ['Mato Grosso', 'MT', 'Centro-Oeste'], ['Mato Grosso do Sul', 'MS', 'Centro-Oeste'], ['Minas Gerais', 'MG', 'Sudeste'],
                          ['Pará', 'PA', 'Norte'], ['Paraíba', 'PB', 'Nordeste'], ['Paraná', 'PR', 'Sul'], ['Pernambuco', 'PE', 'Nordeste'], 
                          ['Piauí', 'PI', 'Nordeste'], ['Rio de Janeiro', 'RJ', 'Sudeste'], ['Rio Grande do Norte', 'RN', 'Nordeste'], 
                          ['Rio Grande do Sul', 'RS', 'Sul'], ['Rondônia', 'RO', 'Norte'], ['Roraima', 'RR', 'Norte'], ['Santa Catarina', 'SC', 'Sul'],
                          ['São Paulo', 'SP', 'Sudeste'], ['Sergipe', 'SE', 'Nordeste'], ['Tocantins', 'TO', 'Norte']],
                        columns=['estado', 'uf', 'região'])
proposicoes1 = pd.merge(proposicoes2, siglas_df, on='uf')
proposicoes1.head()

,id_proposicao,id_deputado,politico,partido,uf,ano,tipo,ementa,palavras_chave,status,data,estado,região
0,2343468,204457,juninho do pneu,UNIÃO,RJ,2019,Parecer do Relator,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",NaN,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",2019-07-03,Rio de Janeiro,Sudeste
1,2237302,141450,hugo leal,PSD,RJ,2019,Emenda ao Substitutivo,Suprima-se a art. 115 do Substitutivo ao Proje...,NaN,Apresentação da Emenda ao Substitutivo n. 84 P...,2019-12-10,Rio de Janeiro,Sudeste
2,2237301,141450,hugo leal,PSD,RJ,2019,Emenda ao Substitutivo,Acrescenta-se o Parágrafo único ao art. 64 con...,NaN,Apresentação da Emenda ao Substitutivo n. 83 P...,2019-12-10,Rio de Janeiro,Sudeste
3,2237300,141450,hugo leal,PSD,RJ,2019,Emenda ao Substitutivo,Acrescenta-se o art. 101 constante do art. 1º ...,NaN,Apresentação da Emenda ao Substitutivo n. 82 P...,2019-12-10,Rio de Janeiro,Sudeste
4,2237259,141450,hugo leal,PSD,RJ,2019,Emenda ao Substitutivo,"Altera a Lei nº 9.503, de 23 de setembro de 19...",NaN,Apresentação da Emenda ao Substitutivo n. 29 P...,2019-12-04,Rio de Janeiro,Sudeste


In [ ]:
list(proposicoes1['politico'].unique())

['juninho do pneu',
 'hugo leal',
 'aureo ribeiro',
 'helio lopes',
 'alessandro molon',
 'paulo ganime',
 'gurgel',
 'daniel silveira',
 'daniela do waguinho',
 'marcelo calero',
 'márcio labre',
 'benedita da silva',
 'glauber braga',
 'marcelo freixo',
 'david miranda',
 'luiz lima',
 'rosangela gomes',
 'sóstenes cavalcante',
 'soraya santos',
 'otoni de paula',
 'chris tonietto',
 'jorge braz',
 'vinicius farah',
 'chico dangelo',
 'talíria petrone',
 'paulo ramos',
 'alexandre serfiotis',
 'dr luiz antonio teixeira jr',
 'christino aureo',
 'wladimir garotinho',
 'flordelis',
 'jandira feghali',
 'felício laterça',
 'clarissa garotinho',
 'professor joziel',
 'pedro paulo',
 'delegado antônio furtado',
 'major fabiana',
 'chiquinho brazão',
 'gutemberg reis',
 'lourival gomes',
 'luiz antônio corrêa',
 'carlos jordy',
 'gelson azevedo',
 'altineu côrtes',
 'ricardo pericar',
 'rodrigo maia',
 'aureo',
 'chico alencar',
 'laura carneiro',
 'marcão gomes',
 'doutor luizinho',
 'jon

### Reeleitos

In [ ]:
reeleitos = pd.DataFrame([['renildo calheiros', 'Reeleito(a)'], ['luiza erundina', 'Reeleito(a)'], ['reginaldo lopes', 'Reeleito(a)'], ['andré figueiredo', 'Reeleito(a)'], 
                          ['altineu côrtes', 'Reeleito(a)'], ['isnaldo bulhões jr', 'Reeleito(a)'], ['sâmia bomfim', 'Reeleito(a)'], ['elmar nascimento', 'Reeleito(a)'], 
                          ['andré fufuca', 'Reeleito(a)'], ['talíria petrone', 'Reeleito(a)'], ['antonio brito', 'Reeleito(a)'], ['ricardo barros', 'Reeleito(a)'],
                          ['celso russomanno', 'Reeleito(a)'], ['vinicius carvalho', 'Reeleito(a)'], ['leandre', 'Reeleito(a)'], ['toninho wandscheer', 'Reeleito(a)'],
                          ['euclydes pettersen', 'Reeleito(a)'], ['vermelho', 'Reeleito(a)'], ['igor timo', 'Reeleito(a)'], ['coronel chrisóstomo', 'Reeleito(a)'], 
                          ['lídice da mata', 'Reeleito(a)'], ['otoni de paula', 'Reeleito(a)'], ['paulo teixeira', 'Reeleito(a)'], ['nilto tatto', 'Reeleito(a)'], 
                          ['erika kokay', 'Reeleito(a)'], ['fernanda melchionna', 'Reeleito(a)'], ['eduardo da fonte', 'Reeleito(a)'], ['flávia morais', 'Reeleito(a)'],
                          ['carmen zanotto', 'Reeleito(a)'], ['helder salomão', 'Reeleito(a)'], ['da vitoria', 'Reeleito(a)'], ['josé nelto', 'Reeleito(a)'], 
                          ['pedro westphalen', 'Reeleito(a)'], ['doutor luizinho', 'Reeleito(a)'], ['daniel freitas', 'Reeleito(a)'], ['luisa canziani', 'Reeleito(a)'], 
                          ['daniela do waguinho', 'Reeleito(a)'], ['claudio cajado', 'Reeleito(a)'], ['diego garcia', 'Reeleito(a)'], ['acácio favacho', 'Reeleito(a)'], 
                          ['túlio gadêlha', 'Reeleito(a)'], ['felipe carreras', 'Reeleito(a)'], ['paulo foletto', 'Reeleito(a)'], ['augusto coutinho', 'Reeleito(a)'],
                          ['natália bonavides', 'Reeleito(a)'], ['benes leocádio', 'Reeleito(a)'], ['adolfo viana', 'Reeleito(a)'], ['jandira feghali', 'Reeleito(a)'], 
                          ['gilson marques', 'Reeleito(a)'], ['josé medeiros', 'Reeleito(a)'], ['sergio souza', 'Reeleito(a)'], ['pastor gil', 'Reeleito(a)'], 
                          ['luiz lima', 'Reeleito(a)'], ['chris tonietto', 'Reeleito(a)'], ['alex manente', 'Reeleito(a)'], ['junio amaral', 'Reeleito(a)'], 
                          ['kim kataguiri', 'Reeleito(a)'], ['rejane dias', 'Reeleito(a)'], ['aureo ribeiro', 'Reeleito(a)'], ['gervásio maia', 'Reeleito(a)'], 
                          ['marcel van hattem', 'Reeleito(a)'], ['afonso motta', 'Reeleito(a)'], ['adriana ventura', 'Reeleito(a)'], ['tabata amaral', 'Reeleito(a)'], 
                          ['lucas redecker', 'Reeleito(a)'], ['alceu moreira', 'Reeleito(a)'], ['silvia cristina', 'Reeleito(a)'], ['zé vitor', 'Reeleito(a)'], 
                          ['eduardo bismarck', 'Reeleito(a)'], ['capitão alberto neto', 'Reeleito(a)'], ['alencar santana', 'Reeleito(a)'], ['aluisio mendes', 'Reeleito(a)'],
                          ['patrus ananias', 'Reeleito(a)'], ['maria do rosário', 'Reeleito(a)'], ['joseildo ramos', 'Reeleito(a)'], ['rosana valle', 'Reeleito(a)'],
                          ['beto pereira', 'Reeleito(a)'], ['márcio jerry', 'Reeleito(a)'], ['carlos veras', 'Reeleito(a)'], ['delegado marcelo freitas', 'Reeleito(a)'], 
                          ['felipe francischini', 'Reeleito(a)'], ['airton faleiro', 'Reeleito(a)'], ['weliton prado', 'Reeleito(a)'], ['giovani cherini', 'Reeleito(a)'],
                          ['covatti filho', 'Reeleito(a)'], ['dr luiz ovando', 'Reeleito(a)'], ['raimundo costa', 'Reeleito(a)'], ['greyce elias', 'Reeleito(a)'],
                          ['eduardo bolsonaro', 'Reeleito(a)'], ['sargento fahur', 'Reeleito(a)'], ['joão daniel', 'Reeleito(a)'], ['maria rosas', 'Reeleito(a)'], 
                          ['carlos chiodini', 'Reeleito(a)'], ['marcio alvino', 'Reeleito(a)'], ['capitão augusto', 'Reeleito(a)'], ['jefferson campos', 'Reeleito(a)'],
                          ['jorge braz', 'Reeleito(a)'], ['luizianne lins', 'Reeleito(a)'], ['hugo leal', 'Reeleito(a)'], ['lincoln portela', 'Reeleito(a)'],
                          ['cezinha de madureira', 'Reeleito(a)'], ['enio verri', 'Reeleito(a)'], ['ricardo silva', 'Reeleito(a)'], ['aj albuquerque', 'Reeleito(a)'],
                          ['arlindo chinaglia', 'Reeleito(a)'], ['sanderson', 'Reeleito(a)'], ['pompeo de mattos', 'Reeleito(a)'], ['pedro paulo', 'Reeleito(a)'],
                          ['sidney leite', 'Reeleito(a)'], ['pedro lupion', 'Reeleito(a)'], ['carlos gomes', 'Reeleito(a)'], ['diego andrade', 'Reeleito(a)'], 
                          ['dr zacharias calil', 'Reeleito(a)'], ['cleber verde', 'Reeleito(a)'], ['júlio cesar', 'Reeleito(a)'], ['lucio mosquini', 'Reeleito(a)'],
                          ['paulão', 'Reeleito(a)'], ['célio silveira', 'Reeleito(a)'], ['ruy carneiro', 'Reeleito(a)'], ['rogério correia', 'Reeleito(a)'],
                          ['sóstenes cavalcante', 'Reeleito(a)'], ['glaustin da fokus', 'Reeleito(a)'], ['osmar terra', 'Reeleito(a)'], ['vicentinho júnior', 'Reeleito(a)'],
                          ['carlos jordy', 'Reeleito(a)'], ['bia kicis', 'Reeleito(a)'], ['general girão', 'Reeleito(a)'], ['josé rocha', 'Reeleito(a)'], 
                          ['joaquim passarinho', 'Reeleito(a)'], ['gilberto abramo', 'Reeleito(a)'], ['juscelino filho', 'Reeleito(a)'], ['zé silva', 'Reeleito(a)'],
                          ['silas câmara', 'Reeleito(a)'], ['eli borges', 'Reeleito(a)'], ['aliel machado', 'Reeleito(a)'], ['marco bertaiolli', 'Reeleito(a)'],
                          ['david soares', 'Reeleito(a)'], ['flávio nogueira', 'Reeleito(a)'], ['alice portugal', 'Reeleito(a)'], ['pedro lucas fernandes', 'Reeleito(a)'],
                          ['ricardo guidi', 'Reeleito(a)'], ['padre joão', 'Reeleito(a)'], ['alexandre padilha', 'Reeleito(a)'], ['soraya santos', 'Reeleito(a)'],
                          ['renata abreu', 'Reeleito(a)'], ['paulo pimenta', 'Reeleito(a)'], ['leonardo monteiro', 'Reeleito(a)'], ['vander loubet', 'Reeleito(a)'], 
                          ['bohn gass', 'Reeleito(a)'], ['marcon', 'Reeleito(a)'], ['paulo guedes', 'Reeleito(a)'], ['afonso florence', 'Reeleito(a)'], ['valmir assunção', 'Reeleito(a)'],
                          ['waldenor pereira', 'Reeleito(a)'], ['jorge solla', 'Reeleito(a)'], ['benedita da silva', 'Reeleito(a)'], ['odair cunha', 'Reeleito(a)'], 
                          ['rubens otoni', 'Reeleito(a)'], ['carlos zarattini', 'Reeleito(a)'], ['rui falcão', 'Reeleito(a)'], ['zé neto', 'Reeleito(a)'], ['pastor eurico', 'Reeleito(a)'],
                          ['fred costa', 'Reeleito(a)'], ['juninho do pneu', 'Reeleito(a)'], ['danilo forte', 'Reeleito(a)'], ['nicoletti', 'Reeleito(a)'], ['arnaldo jardim', 'Reeleito(a)'], 
                          ['luciano ducci', 'Reeleito(a)'], ['heitor schuch', 'Reeleito(a)'], ['carla zambelli', 'Reeleito(a)'], ['andré janones', 'Reeleito(a)'],
                          ['josivaldo jp', 'Reeleito(a)'], ['marcos aurélio sampaio', 'Reeleito(a)'], ['wellington roberto', 'Reeleito(a)'], ['gilberto nascimento', 'Reeleito(a)'],
                          ['rodrigo de castro', 'Reeleito(a)'], ['domingos neto', 'Reeleito(a)'], ['eros biondini', 'Reeleito(a)'], ['luis tibé', 'Reeleito(a)'], 
                          ['jhonatan de jesus', 'Reeleito(a)'], ['luiz nishimori', 'Reeleito(a)'], ['stefano aguiar', 'Reeleito(a)'], ['marx beltrão', 'Reeleito(a)'], 
                          ['misael varella', 'Reeleito(a)'], ['newton cardoso jr', 'Reeleito(a)'], ['fausto pinato', 'Reeleito(a)'], ['miguel lombardi', 'Reeleito(a)'], 
                          ['moses rodrigues', 'Reeleito(a)'], ['celso sabino', 'Reeleito(a)'], ['júnior ferrari', 'Reeleito(a)'], ['silvio costa filho', 'Reeleito(a)'], 
                          ['fabio schiochet', 'Reeleito(a)'], ['fernando rodolfo', 'Reeleito(a)'], ['pinheirinho', 'Reeleito(a)'], ['chiquinho brazão', 'Reeleito(a)'], 
                          ['joão carlos bacelar', 'Reeleito(a)'], ['márcio marinho', 'Reeleito(a)'], ['mário negromonte jr', 'Reeleito(a)'], ['filipe barros', 'Reeleito(a)'], 
                          ['bacelar', 'Reeleito(a)'], ['vitor lippi', 'Reeleito(a)'], ['luciano bivar', 'Reeleito(a)'], ['dimas fabiano', 'Reeleito(a)'], 
                          ['zeca dirceu', 'Reeleito(a)'], ['alexandre leite', 'Reeleito(a)'], ['hercílio coelho diniz', 'Reeleito(a)'], ['adriano do baldy', 'Reeleito(a)'], 
                          ['marreca filho', 'Reeleito(a)'], ['baleia rossi', 'Reeleito(a)'], ['afonso hamm', 'Reeleito(a)'], ['domingos sávio', 'Reeleito(a)'], 
                          ['juarez costa', 'Reeleito(a)'], ['milton vieira', 'Reeleito(a)'], ['joão maia', 'Reeleito(a)'], ['giacobo', 'Reeleito(a)'], ['carlos sampaio', 'Reeleito(a)'],
                          ['sandro alex', 'Reeleito(a)'], ['magda mofatto', 'Reeleito(a)'], ['delegado éder mauro', 'Reeleito(a)'], ['márcio biolchi', 'Reeleito(a)'], 
                          ['dr jaziel', 'Reeleito(a)'], ['amaro neto', 'Reeleito(a)'], ['professor alcides', 'Reeleito(a)'], ['emidinho madeira', 'Reeleito(a)'], 
                          ['andré ferreira', 'Reeleito(a)'], ['marcelo moraes', 'Reeleito(a)'], ['dr frederico', 'Reeleito(a)'], ['capitão derrite', 'Reeleito(a)'],
                          ['alex santana', 'Reeleito(a)'], ['marcelo álvaro antônio', 'Reeleito(a)'], ['wilson santiago', 'Reeleito(a)'], ['fabio reis', 'Reeleito(a)'], 
                          ['pastor sargento isidório', 'Reeleito(a)'], ['célio studart', 'Reeleito(a)'], ['robério monteiro', 'Reeleito(a)'], ['júnior mano', 'Reeleito(a)'], 
                          ['sérgio brito', 'Reeleito(a)'], ['hugo motta', 'Reeleito(a)'], ['rosangela gomes', 'Reeleito(a)'], ['gutemberg reis', 'Reeleito(a)'],
                          ['átila lins', 'Reeleito(a)'], ['josimar maranhãozinho', 'Reeleito(a)'], ['lázaro botelho', 'Reeleito(a)'], ['olival marques', 'Reeleito(a)'],
                          ['gustinho ribeiro', 'Reeleito(a)'], ['marcos pereira', 'Reeleito(a)'], ['junior lourenço', 'Reeleito(a)'], ['aécio neves', 'Reeleito(a)'], 
                          ['fernando monteiro', 'Reeleito(a)'], ['antônia lúcia', 'Reeleito(a)'], ['paulo azi', 'Reeleito(a)'], ['paulo magalhães', 'Reeleito(a)'],
                          ['aguinaldo ribeiro', 'Reeleito(a)'], ['vinicius gurgel', 'Reeleito(a)'], ['tiririca', 'Reeleito(a)'], ['julio lopes', 'Reeleito(a)'], 
                          ['arthur lira', 'Reeleito(a)'], ['laura carneiro', 'Reeleito(a)'], ['jorge goetten', 'Reeleito(a)'], ['bebeto', 'Reeleito(a)'], ['celina leão', 'Não reeleito'],
                          ['bira do pindaré', 'Não reeleito'], ['iracema portella', 'Não reeleito'], ['paula belmonte', 'Não reeleito'], ['andré de paula', 'Não reeleito'], 
                          ['dr leonardo', 'Não reeleito'], ['carla dickson', 'Não reeleito'], ['tiago mitraud', 'Não reeleito'], ['margarete coelho', 'Não reeleito'],
                          ['alessandro molon', 'Não reeleito'], ['tadeu alencar', 'Não reeleito'], ['áurea carolina', 'Não reeleito'], ['elcione barbalho', 'Reeleito(a)'], 
                          ['lauriete', 'Não reeleito'], ['christiane de souza yared', 'Não reeleito'], ['dulce miranda', 'Não reeleito'], ['norma ayub', 'Não reeleito'],
                          ['christino aureo', 'Não reeleito'], ['hildo rocha', 'Não reeleito'], ['neucimar fraga', 'Não reeleito'], ['enrico misasi', 'Não reeleito'], 
                          ['darci de matos', 'Não reeleito'], ['pedro vilela', 'Não reeleito'], ['joenia wapichana', 'Não reeleito'], ['denis bezerra', 'Não reeleito'],
                          ['jones moura', 'Não reeleito'], ['bosco costa', 'Não reeleito'], ['luiz carlos motta', 'Não reeleito'], ['bibo nunes', 'Não reeleito'], 
                          ['alan rick', 'Não reeleito'], ['sergio toledo', 'Não reeleito'], ['aline sleutjes', 'Não reeleito'], ['átila lira', 'Não reeleito'], 
                          ['lucas vergilio', 'Não reeleito'], ['vinicius poit', 'Não reeleito'], ['alexis fonteyne', 'Não reeleito'], ['professor joziel', 'Não reeleito'],
                          ['hiran gonçalves', 'Não reeleito'], ['jerônimo goergen', 'Não reeleito'], ['subtenente gonzaga', 'Não reeleito'], ['daniel silveira', 'Não reeleito'],
                          ['rodrigo agostinho', 'Não reeleito'], ['rubens pereira júnior', 'Reeleito(a)'], ['coronel tadeu', 'Não reeleito'], ['delegado pablo', 'Não reeleito'],
                          ['eduardo cury', 'Não reeleito'], ['joão campos', 'Não reeleito'], ['evair vieira de melo', 'Reeleito(a)'], ['lafayette de andrada', 'Reeleito(a)'],
                          ['gurgel', 'Não reeleito'], ['luizão goulart', 'Não reeleito'], ['osires damaso', 'Não reeleito'], ['caroline de toni', 'Não reeleito'],
                          ['josé guimarães', 'Reeleito(a)'], ['orlando silva', 'Não reeleito'], ['felipe rigoni', 'Não reeleito'], ['daniel almeida', 'Não reeleito'], 
                          ['julio cesar ribeiro', 'Não reeleito'], ['bosco saraiva', 'Não reeleito'], ['marcelo castro', 'Não reeleito'], ['sebastião oliveira', 'Não reeleito'],
                          ['tereza nelma', 'Não reeleito'], ['-', 'Não reeleito'], ['vivi reis', 'Não reeleito'], ['liziane bayer', 'Não reeleito'], ['ossesio silva', 'Não reeleito'],
                          ['paulo bengtson', 'Não reeleito'], ['gil cutrim', 'Não reeleito'], ['ivan valente', 'Não reeleito'], ['carlos henrique gaguim', 'Não reeleito'],
                          ['general peternelli', 'Não reeleito'], ['rubens bueno', 'Não reeleito'], ['luiz philippe de orleans e bragança', 'Não reeleito'], 
                          ['eli corrêa filho', 'Não reeleito'], ['josé priante', 'Reeleito(a)'], ['jesus sérgio', 'Não reeleito'], ['otto alencar filho', 'Reeleito(a)'],
                          ['severino pessoa', 'Não reeleito'], ['professora dorinha seabra rezende', 'Não reeleito'], ['mário heringer', 'Reeleito(a)'], 
                          ['aelton freitas', 'Não reeleito'], ['júlio delgado', 'Não reeleito'], ['jaqueline cassol', 'Não reeleito'], ['nelson barbudo', 'Não reeleito'],
                          ['cristiano vale', 'Não reeleito'], ['mara rocha', 'Não reeleito'], ['coronel armando', 'Não reeleito'], ['jose mario schreiner', 'Não reeleito'],
                          ['milton coelho', 'Não reeleito'], ['marcelo calero', 'Não reeleito'], ['carlos bezerra', 'Não reeleito'], ['glauber braga', 'Não reeleito'], 
                          ['alexandre frota', 'Não reeleito'], ['ney leprevost', 'Não reeleito'], ['frei anastacio ribeiro', 'Não reeleito'], ['pedro augusto bezerra', 'Não reeleito'],
                          ['leur lomanto júnior', 'Reeleito(a)'], ['merlong solano', 'Não reeleito'], ['leônidas cristino', 'Não reeleito'], ['aline gurgel', 'Não reeleito'],
                          ['perpétua almeida', 'Não reeleito'], ['josé ricardo', 'Não reeleito'], ['professora marcivania', 'Não reeleito'], ['camilo capiberibe', 'Não reeleito'], 
                          ['professora rosa neide', 'Não reeleito'], ['geninho zuliani', 'Não reeleito'], ['gustavo fruet', 'Não reeleito'], ['vicentinho', 'Não reeleito'],
                          ['josé airton félix cirilo', 'Reeleito(a)'], ['pedro uczai', 'Não reeleito'], ['marília arraes', 'Não reeleito'], ['célio moura', 'Não reeleito'], 
                          ['leo de brito', 'Não reeleito'], ['damião feliciano', 'Reeleito(a)'], ['henrique fontana', 'Não reeleito'], ['beto faro', 'Não reeleito'],
                          ['gleisi hoffmann', 'Não reeleito'], ['márcio macêdo', 'Não reeleito'], ['francisco jr', 'Não reeleito'], ['charlles evangelista', 'Não reeleito'],
                          ['expedito netto', 'Não reeleito'], ['alê silva', 'Não reeleito'], ['dra soraya manato', 'Não reeleito'], ['osmar serraglio', 'Não reeleito'],
                          ['edna henrique', 'Não reeleito'], ['ricardo izar', 'Não reeleito'], ['eduardo barbosa', 'Não reeleito'], ['angela amin', 'Não reeleito'], 
                          ['luis miranda', 'Não reeleito'], ['celso maldaner', 'Não reeleito'], ['dagoberto nogueira', 'Reeleito(a)'], ['mauro nazif', 'Não reeleito'], 
                          ['maurício dziedricki', 'Não reeleito'], ['pedro cunha lima', 'Não reeleito'], ['daniel coelho', 'Não reeleito'], ['ricardo teobaldo', 'Não reeleito'],
                          ['paulo eduardo martins', 'Não reeleito'], ['léo moraes', 'Não reeleito'], ['daniel trzeciak', 'Não reeleito'], ['guiga peixoto', 'Não reeleito'], 
                          ['lucas gonzalez', 'Não reeleito'], ['paulo ganime', 'Não reeleito'], ['hélio costa', 'Não reeleito'], ['leda sadala', 'Não reeleito'], 
                          ['joice hasselmann', 'Não reeleito'], ['professora dayane pimentel', 'Não reeleito'], ['felício laterça', 'Não reeleito'], ['gonzaga patriota', 'Não reeleito'],
                          ['wolney queiroz', 'Não reeleito'], ['nilson pinto', 'Não reeleito'], ['bilac pinto', 'Não reeleito'], ['efraim filho', 'Não reeleito'],
                          ['paulinho da força', 'Não reeleito'], ['vanderlei macris', 'Não reeleito'], ['luiz carlos', 'Não reeleito'], ['fábio trad', 'Não reeleito'],
                          ['danilo cabral', 'Não reeleito'], ['rogério peninha mendonça', 'Não reeleito'], ['paulo freire costa', 'Não reeleito'], ['roberto de lucena', 'Não reeleito'], 
                          ['andré abdon', 'Não reeleito'], ['hélio leite', 'Não reeleito'], ['mariana carvalho', 'Não reeleito'], ['fábio mitidieri', 'Não reeleito'], 
                          ['marcelo aro', 'Não reeleito'], ['dra vanda milani', 'Não reeleito'], ['vaidon oliveira', 'Não reeleito'], ['lourival gomes', 'Não reeleito'],
                          ['eduardo costa', 'Não reeleito'], ['delegado antônio furtado', 'Não reeleito'], ['haroldo cathedral', 'Não reeleito'], ['fábio henrique', 'Não reeleito'],
                          ['ottaci nascimento', 'Não reeleito'], ['josias gomes', 'Não reeleito'], ['félix mendonça júnior', 'Reeleito(a)'], ['arthur oliveira maia', 'Reeleito(a)'],
                          ['uldurico junior', 'Não reeleito'], ['abílio santana', 'Não reeleito'], ['marcelo ramos', 'Não reeleito'], ['marcelo nilo', 'Não reeleito'], 
                          ['edilázio júnior', 'Não reeleito'], ['tito', 'Não reeleito'], ['charles fernandes', 'Não reeleito'], ['mauro lopes', 'Não reeleito'], 
                          ['elias vaz', 'Não reeleito'], ['tereza cristina', 'Não reeleito'], ['heitor freire', 'Não reeleito'], ['capitão fábio abreu', 'Não reeleito'],
                          ['roberto alves', 'Não reeleito'], ['emanuel pinheiro neto', 'Reeleito(a)'], ['cássio andrade', 'Não reeleito'], ['professor israel batista', 'Não reeleito'],
                          ['chico dangelo', 'Não reeleito'], ['helio lopes', 'Não reeleito'], ['paulo abiackel', 'Reeleito(a)'], ['tiago dimas', 'Não reeleito'], 
                          ['pr marco feliciano', 'Não reeleito'], ['delegado waldir', 'Não reeleito'], ['clarissa garotinho', 'Não reeleito'], ['geovania de sá', 'Não reeleito'], 
                          ['herculano passos', 'Não reeleito'], ['capitão wagner', 'Não reeleito'], ['rose modesto', 'Não reeleito'], ['franco cartafina', 'Não reeleito'],
                          ['márcio labre', 'Não reeleito'], ['major vitor hugo', 'Não reeleito'], ['major fabiana', 'Não reeleito'], ['abou anni', 'Não reeleito'], 
                          ['policial katia sastre', 'Não reeleito'], ['aroldo martins', 'Não reeleito'], ['léo motta', 'Não reeleito'], ['joão roma', 'Não reeleito'],
                          ['beto rosado', 'Não reeleito'], ['onyx lorenzoni', 'Não reeleito'], ['fábio ramalho', 'Não reeleito'], ['marina santos', 'Não reeleito'], 
                          ['edio lopes', 'Não reeleito'], ['paulo ramos', 'Não reeleito'], ['cacá leão', 'Não reeleito'], ['rafael motta', 'Não reeleito'], 
                          ['laercio oliveira', 'Não reeleito'], ['ronaldo martins', 'Não reeleito'], ['vilson da fetaemg', 'Não reeleito'], ['bozzella', 'Não reeleito'],
                          ['rogério carvalho', 'Não reeleito'], ['luiz antônio corrêa', 'Não reeleito'], ['ronaldo carletto', 'Não reeleito'], ['eliza virgínia', 'Não reeleito'],
                          ['loester trutis', 'Não reeleito'], ['neri geller', 'Não reeleito'], ['flaviano melo', 'Não reeleito'], ['mauro benevides filho', 'Reeleito(a)'], 
                          ['vinicius farah', 'Não reeleito'], ['zé carlos', 'Não reeleito'], ['shéridan', 'Não reeleito'], ['rodrigo maia', 'Não reeleito'], 
                          ['genecias noronha', 'Não reeleito'], ['walter alves', 'Não reeleito'], ['joão marcelo souza', 'Não reeleito'], ['ricardo da karol', 'Não reeleito'], 
                          ['idilvan alencar', 'Reeleito(a)'], ['nereu crispim', 'Não reeleito'], ['marlon santos', 'Não reeleito'], ['patricia ferraz', 'Não reeleito'],
                          ['hermes parcianello', 'Não reeleito'], ['raul henry', 'Não reeleito'], ['giovani feltes', 'Não reeleito'], ['alcides rodrigues', 'Não reeleito'],
                          ['alexandre figueiredo', 'Não reeleito'], ['sargento alexandre', 'Não reeleito'], ['tiago andrino', 'Não reeleito'], ['joceval rodrigues', 'Não reeleito'],
                          ['lucas follador', 'Não reeleito'], ['dr luiz antonio teixeira jr', 'Não reeleito'], ['marco brasil', 'Não reeleito'], ['igor kannário', 'Não reeleito'], 
                          ['flávia arruda', 'Não reeleito'], ['eleuses paiva', 'Não reeleito'], ['samuel moreira', 'Não reeleito'], ['danrlei de deus hinterholz', 'Não reeleito'], 
                          ['david miranda', 'Não reeleito'], ['rrenato queiroz', 'Não reeleito'], ['davi alcolumbre', 'Não reeleito'], ['vitor hugo', 'Não reeleito'],
                          ['valtenir pereira', 'Não reeleito'], ['paulo marinho jr', 'Não reeleito'], ['marcos soares', 'Não reeleito'], ['marcelo freixo', 'Não reeleito'],
                          ['vavá martins', 'Não reeleito'], ['gelson azevedo', 'Não reeleito'], ['paes landim', 'Não reeleito'], ['victor mendes', 'Não reeleito'], 
                          ['nelho bezerra', 'Não reeleito'], ['jéssica sales', 'Não reeleito'], ['nivaldo albuquerque', 'Não reeleito'], ['dr agripino magalhães', 'Não reeleito'], 
                          ['pedro augusto palareti', 'Não reeleito'], ['guilherme mussi', 'Não reeleito'], ['otavio leite', 'Não reeleito'], ['julian lemos', 'Não reeleito'],
                          ['leonardo gadelha', 'Não reeleito'], ['alexandre silveira', 'Não reeleito'], ['tia eron', 'Não reeleito'], ['jorielson', 'Não reeleito'], 
                          ['vilalba', 'Não reeleito'], ['patrick dorneles', 'Não reeleito'], ['gastão vieira', 'Não reeleito'], ['roman', 'Não reeleito'], 
                          ['reinhold stephanes junior', 'Não reeleito'], ['ted conti', 'Não reeleito'], ['paulo vicente caleffi', 'Não reeleito'], 
                          ['valdevan noventa', 'Não reeleito'], ['alencar santana braga', 'Não reeleito'], ['bia cavassa', 'Não reeleito'], ['ely santos', 'Não reeleito'],
                          ['marcelo brum', 'Não reeleito'], ['josé nunes', 'Não reeleito'], ['pedro dalua', 'Não reeleito'], ['fernando coelho filho', 'Reeleito(a)'],
                          ['guilherme derrite', 'Não reeleito'], ['bruna furlan', 'Não reeleito'], ['miguel haddad', 'Não reeleito'], ['rossoni', 'Não reeleito'], 
                          ['marcelo squassoni', 'Não reeleito'], ['josé mentor', 'Não reeleito'], ['rose de freitas', 'Não reeleito'], ['paulo pereira da silva', 'Não reeleito'],
                          ['josé aníbal', 'Não reeleito'], ['aníbal gomes', 'Não reeleito'], ['zé augusto nalin', 'Não reeleito'], ['leonardo picciani', 'Não reeleito'], 
                          ['henrique do paraíso', 'Não reeleito'], ['dalua do rota', 'Não reeleito'], ['boca aberta', 'Não reeleito'], ['odorico monteiro', 'Não reeleito'], 
                          ['totonho lopes', 'Não reeleito'], ['rafafá', 'Não reeleito'], ['laerte bessa', 'Não reeleito'], ['flordelis', 'Não reeleito'], ['dr gonçalo', 'Não reeleito'],
                          ['nilda gondim', 'Não reeleito'], ['eduardo gomes', 'Não reeleito'], ['henrique afonso', 'Não reeleito'], ['norma pereira', 'Não reeleito'], 
                          ['schiavinato', 'Não reeleito'], ['gorete pereira', 'Não reeleito'], ['jorge bittar', 'Não reeleito'], ['santini', 'Não reeleito'], 
                          ['marcio bittar', 'Não reeleito'], ['romário', 'Não reeleito'], ['rachel marques', 'Não reeleito'], ['aureo', 'Não reeleito'], ['esperidião amin', 'Não reeleito'],
                          ['marçal filho', 'Não reeleito'], ['dr geraldo resende', 'Reeleito(a)'], ['coronel ulysses', 'Eleito'], ['dr eduardo velloso', 'Eleito'], 
                          ['gerlen diniz', 'Eleito'], ['meire serafim', 'Eleito'], ['roberto duarte', 'Eleito'], ['socorro neri', 'Eleito'], ['zezinho barbary', 'Eleito'], 
                          ['alfredo gaspar', 'Eleito'], ['daniel barbosa', 'Eleito'], ['delegado fabio costa', 'Eleito'], ['luciano amaral', 'Eleito'], ['rafael brito (tio rafa)', 'Eleito'],
                          ['adail filho', 'Eleito'], ['amom mandel', 'Eleito'], ['fausto santos jr', 'Eleito'], ['saullo vianna', 'Eleito'], ['dorinaldo malafaia', 'Eleito'], 
                          ['dr. pupio', 'Eleito'], ['josenildo', 'Eleito'], ['professora goreth', 'Eleito'], ['silvia waiãpi', 'Eleito'], ['sonize barbosa', 'Eleito'],
                          ['capitão alden', 'Eleito'], ['deputado dal', 'Eleito'], ['diego coronel', 'Eleito'], ['gabriel nunes', 'Eleito'], ['ivoneide caetano', 'Eleito'],
                          ['joão leão', 'Eleito'], ['léo prates', 'Eleito'], ['neto carletto', 'Eleito'], ['ricardo maia', 'Eleito'], ['roberta roma', 'Eleito'],
                          ['rogeria santos', 'Eleito'], ['andré fernandes', 'Eleito'], ['dayany do capitão', 'Eleito'], ['eunício', 'Eleito'], ['fernanda pessoa', 'Eleito'],
                          ['luiz gastão', 'Eleito'], ['matheus noronha', 'Eleito'], ['yury do paredão', 'Eleito'], ['fraga', 'Eleito'], ['fred linhares', 'Eleito'], 
                          ['gilvan maximo', 'Eleito'], ['professor reginaldo veras', 'Eleito'], ['rafael prudente', 'Eleito'], ['dr. victor', 'Eleito'], ['gilson daniel', 'Eleito'], 
                          ['gilvan o federal da direita', 'Eleito'], ['jack rocha', 'Eleito'], ['messias donato', 'Eleito'], ['daniel agrobom', 'Eleito'],
                          ['delegada adriana accorsi', 'Eleito'], ['dr ismael alexandrino', 'Eleito'], ['gustavo gayer', 'Eleito'], ['jeferson rodrigues', 'Eleito'],
                          ['lêda borges', 'Eleito'], ['marussa boldrin', 'Eleito'], ['silvye alves', 'Eleito'], ['amanda gentil', 'Eleito'], ['detinha', 'Eleito'], ['duarte', 'Eleito'],
                          ['fábio macedo', 'Eleito'], ['márcio honaiser', 'Eleito'], ['roseana sarney', 'Eleito'], ['ana paula junqueira leao', 'Eleito'], ['ana pimentel', 'Eleito'],
                          ['bruno farias', 'Eleito'], ['célia xakriabá', 'Eleito'], ['dandara', 'Eleito'], ['delegada ione barbosa', 'Eleito'], ['duda salabert', 'Eleito'], 
                          ['luiz fernando', 'Eleito'], ['mauricio do volei', 'Eleito'], ['miguel ângelo', 'Eleito'], ['nely aquino', 'Eleito'], ['nikolas ferreira', 'Eleito'], 
                          ['pedro aihara', 'Eleito'], ['rafael simoes', 'Eleito'], ['rosângela reis', 'Eleito'], ['samuel viana', 'Eleito'], ['camila jara', 'Eleito'], 
                          ['marcos pollon', 'Eleito'], ['rodolfo nogueira', 'Eleito'], ['abílio', 'Eleito'], ['amália barros', 'Eleito'], ['coronel assis', 'Eleito'], 
                          ['coronel fernanda', 'Eleito'], ['fábio garcia', 'Eleito'], ['andreia siqueira', 'Eleito'], ['antonio doido', 'Eleito'], ['delegado caveira', 'Eleito'], 
                          ['dilvanda faro', 'Eleito'], ['dra alessandra haber', 'Eleito'], ['henderson pinto', 'Eleito'], ['keniston', 'Eleito'], ['raimundo santos', 'Eleito'], 
                          ['renilce nicodemos', 'Eleito'], ['cabo gilberto silva', 'Eleito'], ['luiz couto', 'Eleito'], ['mersinho lucena', 'Eleito'], ['murilo galdino', 'Eleito'],
                          ['romero rodrigues', 'Eleito'], ['clarissa tércio', 'Eleito'], ['clodoaldo magalhães', 'Eleito'], ['coronel meira', 'Eleito'], ['eriberto medeiros', 'Eleito'],
                          ['guilherme uchoa junior', 'Eleito'], ['iza arruda', 'Eleito'], ['lucas ramos', 'Eleito'], ['lula da fonte', 'Eleito'], ['maria arraes', 'Eleito'], 
                          ['mendonça filho', 'Eleito'], ['pedro campos', 'Eleito'], ['waldemar oliveira', 'Eleito'], ['atila', 'Eleito'], ['castro neto', 'Eleito'], 
                          ['dr. francisco', 'Eleito'], ['florentino neto', 'Eleito'], ['jadyel da jupi', 'Eleito'], ['julio arcoverde', 'Eleito'], ['beto preto', 'Eleito'], 
                          ['beto richa', 'Eleito'], ['carol dartora', 'Eleito'], ['delegado matheus laiola', 'Eleito'], ['deltan dallagnol', 'Eleito'], ['dilceu sperafico', 'Eleito'], 
                          ['geraldo mendes', 'Eleito'], ['padovani', 'Eleito'], ['paulo litro', 'Eleito'], ['tadeu veneri', 'Eleito'], ['tião medeiros', 'Eleito'], 
                          ['bandeira de mello', 'Eleito'], ['chico alencar', 'Eleito'], ['dani cunha', 'Eleito'], ['daniel soranz', 'Eleito'], ['delegado ramagem', 'Eleito'], 
                          ['dimas gadelha', 'Eleito'], ['general pazuello', 'Eleito'], ['lindbergh farias', 'Eleito'], ['luciano vieira', 'Eleito'], ['marcelo crivella', 'Eleito'],
                          ['marcelo queiroz', 'Eleito'], ['marcos tavares', 'Eleito'], ['max', 'Eleito'], ['murillo gouvea', 'Eleito'], ['pastor henrique vieira', 'Eleito'],
                          ['reimont', 'Eleito'], ['roberto monteiro pai', 'Eleito'], ['sargento portugal', 'Eleito'], ['tarcísio motta', 'Eleito'], ['washington quaquá', 'Eleito'],
                          ['mineiro', 'Eleito'], ['paulinho freire', 'Eleito'], ['robinson faria', 'Eleito'], ['sargento gonçalves', 'Eleito'], ['cristiane lopes', 'Eleito'],
                          ['dr fernando máximo', 'Eleito'], ['lebrão', 'Eleito'], ['maurício carvalho', 'Eleito'], ['thiago flores', 'Eleito'], ['albuquerque', 'Eleito'], 
                          ['defensor stélio dener', 'Eleito'], ['duda ramos', 'Eleito'], ['helena da asatur', 'Eleito'], ['pastor diniz', 'Eleito'], ['zé haroldo cathedral', 'Eleito'],
                          ['alcibio mesquita bibo nunes', 'Eleito'], ['alexandre lindenmeyer', 'Eleito'], ['any ortiz', 'Eleito'], ['busato', 'Eleito'], ['daiana santos', 'Eleito'], 
                          ['denise pessôa', 'Eleito'], ['franciane bayer', 'Eleito'], ['mauricio marcon', 'Eleito'], ['tenente coronel zucco', 'Eleito'], ['ana paula lima', 'Eleito'], 
                          ['cobalchini', 'Eleito'], ['daniela reinehr', 'Eleito'], ['ismael', 'Eleito'], ['julia zanatta', 'Eleito'], ['rafael pezenti', 'Eleito'], ['zé trovão', 'Eleito'],
                          ['delegada katarina', 'Eleito'], ['icaro de valmir', 'Eleito'], ['rodrigo valadares', 'Eleito'], ['thiago de joaldo', 'Eleito'], ['yandra de andré', 'Eleito'],
                          ['alberto mourão', 'Eleito'], ['antonio carlos rodrigues', 'Eleito'], ['bruno ganem', 'Eleito'], ['delegado bruno lima', 'Eleito'], ['delegado da cunha', 'Eleito'],
                          ['delegado palumbo', 'Eleito'], ['delegado paulo bilynskyj', 'Eleito'], ['erika hilton', 'Eleito'], ['fabio teruel', 'Eleito'], ['felipe becari', 'Eleito'], 
                          ['guilherme boulos', 'Eleito'], ['jilmar tatto', 'Eleito'], ['jonas donizette', 'Eleito'], ['juliana cardoso', 'Eleito'], ['kiko celeguim', 'Eleito'], 
                          ['luiz marinho', 'Eleito'], ['marangoni', 'Eleito'], ['marcelo lima', 'Eleito'], ['marina silva', 'Eleito'],['mario frias', 'Eleito'], ['mauricio neves', 'Eleito'], 
                          ['paulo alexandre barbosa', 'Eleito'], ['ricardo salles', 'Eleito'], ['rodrigo gambale', 'Eleito'], ['rosângela moro', 'Eleito'], ['simone marquetto', 'Eleito'], 
                          ['sônia guajajara', 'Eleito'], ['alexandre guimarães', 'Eleito'], ['filipe martins', 'Eleito'], ['ricardo ayres', 'Eleito'], ['toinho andrade', 'Eleito'],
                          ['bispo jose', 'Senador'], ['marcio bittar', 'Senador'], ['sergio petecao', 'Senador'], ['fernando collor', 'Senador'], ['renan calheiros', 'Senador'], ['rodrigo cunha', 'Senador'], ['eduardo braga', 'Senador'], ['omar aziz', 'Senador'], ['plinio valerio', 'Senador'], ['davi alcolumbre', 'Senador'], ['lucas barreto', 'Senador'], ['randolfe rodrigues', 'Senador'], ['angelo coronel', 'Senador'], ['jaques wagner', 'Senador'], ['otto alencar', 'Senador'], ['cid gomes', 'Senador'], ['eduardo girao', 'Senador'], ['tasso jereissati', 'Senador'], ['izalci lucas', 'Senador'], ['leila barros', 'Senador'], ['reguffe', 'Senador'], ['fabiano contarato', 'Senador'], ['marcos do val', 'Senador'], ['rose de freitas', 'Senador'], ['jorge kajuru', 'Senador'], ['luiz carlos do carmo', 'Senador'], ['vanderlan cardoso', 'Senador'], ['eliziane gama', 'Senador'], ['roberto rocha', 'Senador'], ['weverton', 'Senador'], ['carlos viana', 'Senador'], ['rodrigo pacheco', 'Senador'], ['nelsinho trad', 'Senador'], ['soraya thronicke', 'Senador'], ['jayme campos', 'Senador'], ['margareth buzetti', 'Senador'], ['wellington fagundes', 'Senador'], ['jader barbalho', 'Senador'], ['paulo rocha', 'Senador'], ['zequinha marinho', 'Senador'], ['daniella ribeiro', 'Senador'], ['nilda gondim', 'Senador'], ['veneziano vital do rego', 'Senador'], ['fernando bezerra coelho', 'Senador'], ['fernando dueire', 'Senador'], ['humberto costa', 'Senador'], ['ciro nogueira', 'Senador'], ['elmano ferrer', 'Senador'], ['marcelo castro', 'Senador'], ['alvaro dias', 'Senador'], ['flavio arns', 'Senador'], ['oriovisto guimaraes', 'Senador'], ['carlos portinho', 'Senador'], ['flavio bolsonaro', 'Senador'], ['romario', 'Senador'], ['jean paul prates', 'Senador'], ['styvenson valentim', 'Senador'], ['zenaide maia', 'Senador'], ['acir gurgacz', 'Senador'], ['confucio moura', 'Senador'], ['dr. samuel araujo', 'Senador'], ['chico rodrigues', 'Senador'], ['mecias de jesus', 'Senador'], ['telmario mota', 'Senador'], ['lasier martins', 'Senador'], ['luis carlos heinze', 'Senador'], ['paulo paim', 'Senador'], ['dario berger', 'Senador'], ['esperidiao amin', 'Senador'], ['ivete da silveira', 'Senador'], ['alessandro vieira', 'Senador'], ['maria do carmo alves', 'Senador'], ['rogerio carvalho', 'Senador'], ['giordano', 'Senador'], ['jose serra', 'Senador'], ['mara gabrilli', 'Senador'], ['eduardo gomes', 'Senador'], ['iraja', 'Senador'], ['katia abreu', 'Senador']],
                         columns=['politico', 'eleição'])
reeleitos

,politico,eleição
0,renildo calheiros,Reeleito(a)
1,luiza erundina,Reeleito(a)
2,reginaldo lopes,Reeleito(a)
3,andré figueiredo,Reeleito(a)
4,altineu côrtes,Reeleito(a)
...,...,...
887,jose serra,Senador
888,mara gabrilli,Senador
889,eduardo gomes,Senador
890,iraja,Senador


# Limpeza da coluna

In [ ]:
proposicoes1.head()

,id_proposicao,id_deputado,politico,partido,uf,ano,tipo,ementa,palavras_chave,status,data,estado,região
0,2343468,204457,juninho do pneu,UNIÃO,RJ,2019,Parecer do Relator,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",NaN,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",2019-07-03,Rio de Janeiro,Sudeste
1,2237302,141450,hugo leal,PSD,RJ,2019,Emenda ao Substitutivo,Suprima-se a art. 115 do Substitutivo ao Proje...,NaN,Apresentação da Emenda ao Substitutivo n. 84 P...,2019-12-10,Rio de Janeiro,Sudeste
2,2237301,141450,hugo leal,PSD,RJ,2019,Emenda ao Substitutivo,Acrescenta-se o Parágrafo único ao art. 64 con...,NaN,Apresentação da Emenda ao Substitutivo n. 83 P...,2019-12-10,Rio de Janeiro,Sudeste
3,2237300,141450,hugo leal,PSD,RJ,2019,Emenda ao Substitutivo,Acrescenta-se o art. 101 constante do art. 1º ...,NaN,Apresentação da Emenda ao Substitutivo n. 82 P...,2019-12-10,Rio de Janeiro,Sudeste
4,2237259,141450,hugo leal,PSD,RJ,2019,Emenda ao Substitutivo,"Altera a Lei nº 9.503, de 23 de setembro de 19...",NaN,Apresentação da Emenda ao Substitutivo n. 29 P...,2019-12-04,Rio de Janeiro,Sudeste


## Corrigir os nomes dos deputados

In [ ]:
proposicoes2 = proposicoes1.replace({'renildo calheiros': 'renildo calheiros', 'luiza erundina': 'luiza erundina', 'reginaldo lopes': 'reginaldo lopes', 
                                  'andré figueiredo': 'andre figueiredo', 'altineu côrtes': 'altineu cortes', 'isnaldo bulhões jr': 'isnaldo bulhoes jr', 
                                  'sâmia bomfim': 'samia bomfim', 'elmar nascimento': 'elmar nascimento', 'andré fufuca': 'andre fufuca', 
                                  'talíria petrone': 'taliria petrone', 'antonio brito': 'antonio brito', 'ricardo barros': 'ricardo barros', 
                                  'celso russomanno': 'celso russomanno', 'vinicius carvalho': 'vinicius carvalho', 'leandre': 'leandre', 
                                  'toninho wandscheer': 'toninho wandscheer', 'euclydes pettersen': 'euclydes pettersen', 'vermelho': 'vermelho', 
                                  'igor timo': 'igor timo', 'coronel chrisóstomo': 'coronel chrisostomo', 'lídice da mata': 'lidice da mata', 
                                  'otoni de paula': 'otoni de paula', 'paulo teixeira': 'paulo teixeira', 'nilto tatto': 'nilto tatto', 'erika kokay': 'erika kokay',
                                  'fernanda melchionna': 'fernanda melchionna', 'edu=ardo da fonte': 'eduardo da fonte', 'flávia morais': 'flavia morais', 
                                  'carmen zanotto': 'carmen zanotto', 'helder salomão': 'helder salomao', 'da vitoria': 'da vitoria', 'josé nelto': 'jose nelto', 
                                  'pedro westphalen': 'pedro westphalen', 'doutor luizinho': 'doutor luizinho', 'daniel freitas': 'daniel freitas', 
                                  'luisa canziani': 'luisa canziani', 'daniela do waguinho': 'daniela do waguinho', 'claudio cajado': 'claudio cajado',
                                  'diego garcia': 'diego garcia', 'acácio favacho': 'acacio favacho', 'túlio gadêlha': 'tulio gadelha', 'felipe carreras': 'felipe carreras',
                                  'paulo foletto': 'paulo foletto', 'augusto coutinho': 'augusto coutinho', 'natália bonavides': 'natalia bonavides', 
                                  'benes leocádio': 'benes leocadio', 'adolfo viana': 'adolfo viana', 'jandira feghali': 'jandira feghali', 'gilson marques': 'gilson marques',
                                  'josé medeiros': 'jose medeiros', 'sergio souza': 'sergio souza', 'pastor gil': 'pastor gil', 'luiz lima': 'luiz lima', 
                                  'chris tonietto': 'chris tonietto', 'alex manente': 'alex manente', 'junio amaral': 'junio amaral', 'kim kataguiri': 'kim kataguiri', 
                                  'rejane dias': 'rejane dias', 'aureo ribeiro': 'aureo ribeiro', 'gervásio maia': 'gervasio maia', 'marcel van hattem': 'marcel van hattem',
                                  'afonso motta': 'afonso motta', 'adriana ventura': 'adriana ventura', 'tabata amaral': 'tabata amaral', 'lucas redecker': 'lucas redecker',
                                  'alceu moreira': 'alceu moreira', 'silvia cristina': 'silvia cristina', 'zé vitor': 'ze vitor', 'eduardo bismarck': 'eduardo bismarck', 
                                  'capitão alberto neto': 'capitao alberto neto', 'alencar santana': 'alencar santana', 'aluisio mendes': 'aluisio mendes', 
                                  'patrus ananias': 'patrus ananias', 'maria do rosário': 'maria do rosario', 'joseildo ramos': 'joseildo ramos', 'rosana valle': 'rosana valle',
                                  'beto pereira': 'beto pereira', 'márcio jerry': 'marcio jerry', 'carlos veras': 'carlos veras', 
                                  'delegado marcelo freitas': 'delegado marcelo freitas', 'felipe francischini': 'felipe francischini', 'airton faleiro': 'airton faleiro',
                                  'weliton prado': 'weliton prado', 'giovani cherini': 'giovani cherini', 'covatti filho': 'covatti filho', 'dr luiz ovando': 'dr luiz ovando', 
                                  'raimundo costa': 'raimundo costa', 'greyce elias': 'greyce elias', 'eduardo bolsonaro': 'eduardo bolsonaro', 'sargento fahur': 'sargento fahur', 
                                  'joão daniel': 'joao daniel', 'maria rosas': 'maria rosas', 'carlos chiodini': 'carlos chiodini', 'marcio alvino': 'marcio alvino', 
                                  'capitão augusto': 'capitao augusto', 'jefferson campos': 'jefferson campos', 'jorge braz': 'jorge braz', 'luizianne lins': 'luizianne lins', 
                                  'hugo leal': 'hugo leal', 'lincoln portela': 'lincoln portela', 'cezinha de madureira': 'cezinha de madureira', 'enio verri': 'enio verri', 
                                  'ricardo silva': 'ricardo silva', 'aj albuquerque': 'aj albuquerque', 'arlindo chinaglia': 'arlindo chinaglia', 'sanderson': 'sanderson', 
                                  'pompeo de mattos': 'pompeo de mattos', 'pedro paulo': 'pedro paulo', 'sidney leite': 'sidney leite', 'pedro lupion': 'pedro lupion', 
                                  'carlos gomes': 'carlos gomes', 'diego andrade': 'diego andrade', 'dr zacharias calil': 'dr zacharias calil', 'cleber verde': 'cleber verde',
                                  'júlio cesar': 'julio cesar', 'lucio mosquini': 'lucio mosquini', 'paulão': 'paulao', 'célio silveira': 'celio silveira', 
                                  'ruy carneiro': 'ruy carneiro', 'rogério correia': 'rogerio correia', 'sóstenes cavalcante': 'sostenes cavalcante', 
                                  'glaustin da fokus': 'glaustin da fokus', 'osmar terra': 'osmar terra', 'vicentinho júnior': 'vicentinho junior', 'carlos jordy': 'carlos jordy', 
                                  'bia kicis': 'bia kicis', 'general girão': 'general girao', 'josé rocha': 'jose rocha', 'joaquim passarinho': 'joaquim passarinho',
                                  'gilberto abramo': 'gilberto abramo', 'juscelino filho': 'juscelino filho', 'zé silva': 'ze silva', 'silas câmara': 'silas camara',
                                  'eli borges': 'eli borges', 'aliel machado': 'aliel machado', 'marco bertaiolli': 'marco bertaiolli', 'david soares': 'david soares',
                                  'flávio nogueira': 'flavio nogueira', 'alice portugal': 'alice portugal', 'pedro lucas fernandes': 'pedro lucas fernandes', 
                                  'ricardo guidi': 'ricardo guidi', 'padre joão': 'padre joao', 'alexandre padilha': 'alexandre padilha', 'soraya santos': 'soraya santos',
                                  'renata abreu': 'renata abreu', 'paulo pimenta': 'paulo pimenta', 'leonardo monteiro': 'leonardo monteiro', 'vander loubet': 'vander loubet', 
                                  'bohn gass': 'bohn gass', 'marcon': 'marcon', 'paulo guedes': 'paulo guedes', 'afonso florence': 'afonso florence', 
                                  'valmir assunção': 'valmir assuncao', 'waldenor pereira': 'waldenor pereira', 'jorge solla': 'jorge solla', 'benedita da silva': 'benedita da silva',
                                  'odair cunha': 'odair cunha', 'rubens otoni': 'rubens otoni', 'carlos zarattini': 'carlos zarattini', 'rui falcão': 'rui falcao',
                                  'zé neto': 'ze neto', 'pastor eurico': 'pastor eurico', 'fred costa': 'fred costa', 'juninho do pneu': 'juninho do pneu', 
                                  'danilo forte': 'danilo forte', 'nicoletti': 'nicoletti', 'arnaldo jardim': 'arnaldo jardim', 'luciano ducci': 'luciano ducci', 
                                  'heitor schuch': 'heitor schuch', 'carla zambelli': 'carla zambelli', 'andré janones': 'andre janones', 'josivaldo jp': 'josivaldo jp',
                                  'marcos aurélio sampaio': 'marcos aurelio sampaio', 'wellington roberto': 'wellington roberto', 'gilberto nascimento': 'gilberto nascimento', 
                                  'rodrigo de castro': 'rodrigo de castro', 'domingos neto': 'domingos neto', 'eros biondini': 'eros biondini', 'luis tibé': 'luis tibe', 
                                  'jhonatan de jesus': 'jhonatan de jesus', 'luiz nishimori': 'luiz nishimori', 'stefano aguiar': 'stefano aguiar', 'marx beltrão': 'marx beltrao', 
                                  'misael varella': 'misael varella', 'newton cardoso jr': 'newton cardoso jr', 'fausto pinato': 'fausto pinato', 'miguel lombardi': 'miguel lombardi', 
                                  'moses rodrigues': 'moses rodrigues', 'celso sabino': 'celso sabino', 'júnior ferrari': 'junior ferrari', 'silvio costa filho': 'silvio costa filho', 
                                  'fabio schiochet': 'fabio schiochet', 'fernando rodolfo': 'fernando rodolfo', 'pinheirinho': 'pinheirinho', 'chiquinho brazão': 'chiquinho brazao',
                                  'joão carlos bacelar': 'joao carlos bacelar', 'márcio marinho': 'marcio marinho', 'mário negromonte jr': 'mario negromonte jr', 
                                   'filipe barros': 'filipe barros', 'bacelar': 'bacelar', 'vitor lippi': 'vitor lippi', 'luciano bivar': 'luciano bivar', 
                                   'dimas fabiano': 'dimas fabiano', 'zeca dirceu': 'zeca dirceu', 'alexandre leite': 'alexandre leite', 'hercílio coelho diniz': 
                                   'hercilio coelho diniz', 'adriano do baldy': 'adriano do baldy', 'marreca filho': 'marreca filho', 'baleia rossi': 'baleia rossi', 
                                   'afonso hamm': 'afonso hamm', 'domingos sávio': 'domingos savio', 'juarez costa': 'juarez costa', 'milton vieira': 'milton vieira', 
                                   'joão maia': 'joao maia', 'giacobo': 'giacobo', 'carlos sampaio': 'carlos sampaio', 'sandro alex': 'sandro alex', 
                                   'magda mofatto': 'magda mofatto', 'delegado éder mauro': 'delegado eder mauro', 'márcio biolchi': 'marcio biolchi', 'dr jaziel': 'dr jaziel', 
                                   'amaro neto': 'amaro neto', 'professor alcides': 'professor alcides', 'emidinho madeira': 'emidinho madeira', 'andré ferreira': 'andre ferreira', 
                                   'marcelo moraes': 'marcelo moraes', 'dr frederico': 'dr frederico', 'capitão derrite': 'capitao derrite', 'alex santana': 'alex santana', 
                                   'marcelo álvaro antônio': 'marcelo alvaro antonio', 'wilson santiago': 'wilson santiago', 'fabio reis': 'fabio reis', 
                                   'pastor sargento isidório': 'pastor sargento isidorio', 'célio studart': 'celio studart', 'robério monteiro': 'roberio monteiro', 
                                    'júnior mano': 'junior mano', 'sérgio brito': 'sergio brito', 'hugo motta': 'hugo motta', 'rosangela gomes': 'rosangela gomes', 
                                    'gutemberg reis': 'gutemberg reis', 'átila lins': 'atila lins', 'josimar maranhãozinho': 'josimar maranhaozinho', 
                                    'lázaro botelho': 'lazaro botelho', 'olival marques': 'olival marques', 'gustinho ribeiro': 'gustinho ribeiro', 
                                    'marcos pereira': 'marcos pereira', 'junior lourenço': 'junior lourenco', 'aécio neves': 'aecio neves', 'fernando monteiro': 'fernando monteiro',
                                    'antônia lúcia': 'antonia lucia', 'paulo azi': 'paulo azi', 'paulo magalhães': 'paulo magalhaes', 'aguinaldo ribeiro': 'aguinaldo ribeiro', 
                                   'vinicius gurgel': 'vinicius gurgel', 'tiririca': 'tiririca', 'julio lopes': 'julio lopes', 'arthur lira': 'arthur lira', 
                                    'laura carneiro': 'laura carneiro', 'jorge goetten': 'jorge goetten', 'bebeto': 'bebeto', 'celina leão': 'celina leao', 
                                   'bira do pindaré': 'bira do pindare', 'iracema portella': 'iracema portella', 'paula belmonte': 'paula belmonte', 
                                   'andré de paula': 'andre de paula', 'dr leonardo': 'dr leonardo', 'carla dickson': 'carla dickson', 'tiago mitraud': 'tiago mitraud',
                                   'margarete coelho': 'margarete coelho', 'alessandro molon': 'alessandro molon', 'tadeu alencar': 'tadeu alencar', 
                                   'áurea carolina': 'aurea carolina', 'elcione barbalho': 'elcione barbalho', 'lauriete': 'lauriete', 
                                   'christiane de souza yared': 'christiane de souza yared', 'dulce miranda': 'dulce miranda', 'norma ayub': 'norma ayub',
                                   'christino aureo': 'christino aureo', 'hildo rocha': 'hildo rocha', 'neucimar fraga': 'neucimar fraga', 'enrico misasi': 'enrico misasi', 
                                   'darci de matos': 'darci de matos', 'pedro vilela': 'pedro vilela', 'joenia wapichana': 'joenia wapichana', 'denis bezerra': 'denis bezerra', 
                                   'jones moura': 'jones moura', 'bosco costa': 'bosco costa', 'luiz carlos motta': 'luiz carlos motta', 'bibo nunes': 'bibo nunes', 
                                   'alan rick': 'alan rick', 'sergio toledo': 'sergio toledo', 'aline sleutjes': 'aline sleutjes', 'átila lira': 'atila lira', 
                                   'lucas vergilio': 'lucas vergilio', 'vinicius poit': 'vinicius poit', 'alexis fonteyne': 'alexis fonteyne', 'professor joziel': 'professor joziel',
                                   'hiran gonçalves': 'hiran goncalves', 'jerônimo goergen': 'jeronimo goergen', 'subtenente gonzaga': 'subtenente gonzaga', 
                                   'daniel silveira': 'daniel silveira', 'rodrigo agostinho': 'rodrigo agostinho', 'rubens pereira júnior': 'rubens pereira junior', 
                                   'coronel tadeu': 'coronel tadeu', 'delegado pablo': 'delegado pablo', 'eduardo cury': 'eduardo cury', 'joão campos': 'joao campos', 
                                   'evair vieira de melo': 'evair vieira de melo', 'lafayette de andrada': 'lafayette de andrada', 'gurgel': 'gurgel', 
                                   'luizão goulart': 'luizao goulart', 'osires damaso': 'osires damaso', 'caroline de toni': 'caroline de toni', 'josé guimarães': 'jose guimaraes',
                                   'orlando silva': 'orlando silva', 'felipe rigoni': 'felipe rigoni', 'daniel almeida': 'daniel almeida', 'julio cesar ribeiro': 'julio cesar ribeiro',
                                   'bosco saraiva': 'bosco saraiva', 'marcelo castro': 'marcelo castro', 'sebastião oliveira': 'sebastiao oliveira', 'tereza nelma': 'tereza nelma', 
                                   'rodrigo coelho': 'rodrigo coelho', 'vivi reis': 'vivi reis', 'liziane bayer': 'liziane bayer', 'ossesio silva': 'ossesio silva', 'paulo bengtson': 'paulo bengtson', 
                                   'gil cutrim': 'gil cutrim', 'ivan valente': 'ivan valente', 'carlos henrique gaguim': 'carlos henrique gaguim', 'general peternelli': 'general peternelli', 
                                   'rubens bueno': 'rubens bueno', 'luiz philippe de orleans e bragança': 'luiz philippe de orleans e braganca', 'eli corrêa filho': 'eli correa filho', 
                                   'josé priante': 'jose priante', 'jesus sérgio': 'jesus sergio', 'otto alencar filho': 'otto alencar filho', 'severino pessoa': 'severino pessoa', 
                                   'professora dorinha seabra rezende': 'professora dorinha seabra rezende', 'mário heringer': 'mario heringer', 'aelton freitas': 'aelton freitas', 
                                   'júlio delgado': 'julio delgado', 'jaqueline cassol': 'jaqueline cassol', 'nelson barbudo': 'nelson barbudo', 'cristiano vale': 'cristiano vale', 
                                   'mara rocha': 'mara rocha', 'coronel armando': 'coronel armando', 'jose mario schreiner': 'jose mario schreiner', 'milton coelho': 'milton coelho', 
                                   'marcelo calero': 'marcelo calero', 'carlos bezerra': 'carlos bezerra', 'glauber braga': 'glauber braga', 'alexandre frota': 'alexandre frota', 
                                   'ney leprevost': 'ney leprevost', 'frei anastacio ribeiro': 'frei anastacio ribeiro', 'pedro augusto bezerra': 'pedro augusto bezerra', 
                                    'leur lomanto júnior': 'leur lomanto junior', 'merlong solano': 'merlong solano', 'leônidas cristino': 'leonidas cristino', 'aline gurgel': 'aline gurgel',
                                   'perpétua almeida': 'perpetua almeida', 'josé ricardo': 'jose ricardo', 'professora marcivania': 'professora marcivania', 'camilo capiberibe': 'camilo capiberibe',
                                   'professora rosa neide': 'professora rosa neide', 'geninho zuliani': 'geninho zuliani', 'gustavo fruet': 'gustavo fruet', 'vicentinho': 'vicentinho',
                                   'josé airton félix cirilo': 'jose airton felix cirilo', 'pedro uczai': 'pedro uczai', 'marília arraes': 'marilia arraes', 'célio moura': 'celio moura', 
                                   'leo de brito': 'leo de brito', 'damião feliciano': 'damiao feliciano', 'henrique fontana': 'henrique fontana', 'beto faro': 'beto faro', 
                                   'gleisi hoffmann': 'gleisi hoffmann', 'márcio macêdo': 'marcio macedo', 'francisco jr': 'francisco jr', 'charlles evangelista': 'charlles evangelista',
                                   'expedito netto': 'expedito netto', 'alê silva': 'ale silva', 'dra soraya manato': 'dra soraya manato', 'osmar serraglio': 'osmar serraglio', 
                                   'edna henrique': 'edna henrique', 'ricardo izar': 'ricardo izar', 'eduardo barbosa': 'eduardo barbosa', 'angela amin': 'angela amin', 
                                   'luis miranda': 'luis miranda', 'celso maldaner': 'celso maldaner', 'dagoberto nogueira': 'dagoberto nogueira', 'mauro nazif': 'mauro nazif',
                                   'maurício dziedricki': 'mauricio dziedricki', 'pedro cunha lima': 'pedro cunha lima', 'daniel coelho': 'daniel coelho', 'ricardo teobaldo': 'ricardo teobaldo', 
                                   'paulo eduardo martins': 'paulo eduardo martins', 'léo moraes': 'leo moraes', 'daniel trzeciak': 'daniel trzeciak', 'guiga peixoto': 'guiga peixoto',
                                   'lucas gonzalez': 'lucas gonzalez', 'paulo ganime': 'paulo ganime', 'hélio costa': 'helio costa', 'leda sadala': 'leda sadala', 'joice hasselmann': 'joice hasselmann', 
                                   'professora dayane pimentel': 'professora dayane pimentel', 'felício laterça': 'felicio laterca', 'gonzaga patriota': 'gonzaga patriota', 
                                   'wolney queiroz': 'wolney queiroz', 'nilson pinto': 'nilson pinto', 'bilac pinto': 'bilac pinto', 'efraim filho': 'efraim filho', 'paulinho da força': 'paulinho da forca',
                                   'vanderlei macris': 'vanderlei macris', 'luiz carlos': 'luiz carlos', 'fábio trad': 'fabio trad', 'danilo cabral': 'danilo cabral',
                                   'rogério peninha mendonça': 'rogerio peninha mendonca', 'paulo freire costa': 'paulo freire costa', 'roberto de lucena': 'roberto de lucena',
                                   'andré abdon': 'andre abdon', 'hélio leite': 'helio leite', 'mariana carvalho': 'mariana carvalho', 'fábio mitidieri': 'fabio mitidieri',
                                   'marcelo aro': 'marcelo aro', 'dra vanda milani': 'dra vanda milani', 'vaidon oliveira': 'vaidon oliveira', 'lourival gomes': 'lourival gomes', 
                                   'eduardo costa': 'eduardo costa', 'delegado antônio furtado': 'delegado antonio furtado', 'haroldo cathedral': 'haroldo cathedral', 'fábio henrique': 'fabio henrique', 
                                   'ottaci nascimento': 'ottaci nascimento', 'josias gomes': 'josias gomes', 'félix mendonça júnior': 'felix mendonca junior', 'arthur oliveira maia': 'arthur oliveira maia', 
                                   'uldurico junior': 'uldurico junior', 'abílio santana': 'abilio santana', 'marcelo ramos': 'marcelo ramos', 'marcelo nilo': 'marcelo nilo', 'edilázio júnior': 'edilazio junior', 
                                   'tito': 'tito', 'charles fernandes': 'charles fernandes', 'mauro lopes': 'mauro lopes', 'elias vaz': 'elias vaz', 'tereza cristina': 'tereza cristina', 
                                   'heitor freire': 'heitor freire', 'capitão fábio abreu': 'capitao fabio abreu', 'roberto alves': 'roberto alves', 'emanuel pinheiro neto': 'emanuel pinheiro neto', 
                                   'cássio andrade': 'cassio andrade', 'professor israel batista': 'professor israel batista', 'chico dangelo': 'chico dangelo', 'helio lopes': 'helio lopes', 
                                   'paulo abiackel': 'paulo abiackel', 'tiago dimas': 'tiago dimas', 'pr marco feliciano': 'pr marco feliciano', 'delegado waldir': 'delegado waldir', 
                                   'clarissa garotinho': 'clarissa garotinho', 'geovania de sá': 'geovania de sa', 'herculano passos': 'herculano passos', 'capitão wagner': 'capitao wagner', 
                                   'rose modesto': 'rose modesto', 'franco cartafina': 'franco cartafina', 'márcio labre': 'marcio labre', 'major vitor hugo': 'major vitor hugo', 
                                   'major fabiana': 'major fabiana', 'abou anni': 'abou anni', 'policial katia sastre': 'policial katia sastre', 'aroldo martins': 'aroldo martins', 
                                    'léo motta': 'leo motta', 'joão roma': 'joao roma', 'beto rosado': 'beto rosado', 'onyx lorenzoni': 'onyx lorenzoni', 'fábio ramalho': 'fabio ramalho', 
                                   'marina santos': 'marina santos', 'edio lopes': 'edio lopes', 'paulo ramos': 'paulo ramos', 'cacá leão': 'caca leao', 'rafael motta': 'rafael motta', 
                                    'laercio oliveira': 'laercio oliveira', 'ronaldo martins': 'ronaldo martins', 'vilson da fetaemg': 'vilson da fetaemg', 'bozzella': 'bozzella', 
                                    'rogério carvalho': 'rogerio carvalho', 'luiz antônio corrêa': 'luiz antonio correa', 'ronaldo carletto': 'ronaldo carletto', 'eliza virgínia': 'eliza virginia',
                                   'loester trutis': 'loester trutis', 'neri geller': 'neri geller', 'flaviano melo': 'flaviano melo', 'mauro benevides filho': 'mauro benevides filho', 
                                    'vinicius farah': 'vinicius farah', 'zé carlos': 'ze carlos', 'shéridan': 'sheridan', 'rodrigo maia': 'rodrigo maia', 'genecias noronha': 'genecias noronha', 
                                    'walter alves': 'walter alves', 'joão marcelo souza': 'joao marcelo souza', 'ricardo da karol': 'ricardo da karol', 'idilvan alencar': 'idilvan alencar', 
                                   'nereu crispim': 'nereu crispim', 'marlon santos': 'marlon santos', 'patricia ferraz': 'patricia ferraz', 'hermes parcianello': 'hermes parcianello', 
                                    'raul henry': 'raul henry', 'giovani feltes': 'giovani feltes', 'alcides rodrigues': 'alcides rodrigues', 'alexandre figueiredo': 'alexandre figueiredo', 
                                    'sargento alexandre': 'sargento alexandre', 'tiago andrino': 'tiago andrino', 'joceval rodrigues': 'joceval rodrigues', 'lucas follador': 'lucas follador', 
                                   'dr luiz antonio teixeira jr': 'dr luiz antonio teixeira jr', 'marco brasil': 'marco brasil', 'igor kannário': 'igor kannario', 'flávia arruda': 'flavia arruda',
                                    'eleuses paiva': 'eleuses paiva', 'samuel moreira': 'samuel moreira', 'danrlei de deus hinterholz': 'danrlei de deus hinterholz', 'david miranda': 'david miranda',
                                    'rrenato queiroz': 'rrenato queiroz', 'davi alcolumbre': 'davi alcolumbre', 'vitor hugo': 'vitor hugo', 'valtenir pereira': 'valtenir pereira', 
                                    'paulo marinho jr': 'paulo marinho jr', 'marcos soares': 'marcos soares', 'marcelo freixo': 'marcelo freixo', 'vavá martins': 'vava martins', 'gelson azevedo': 'gelson azevedo',
                                    'paes landim': 'paes landim', 'victor mendes': 'victor mendes', 'nelho bezerra': 'nelho bezerra', 'jéssica sales': 'jessica sales', 'nivaldo albuquerque': 'nivaldo albuquerque',
                                    'dr agripino magalhães': 'dr agripino magalhaes', 'pedro augusto palareti': 'pedro augusto palareti', 'guilherme mussi': 'guilherme mussi', 'otavio leite': 'otavio leite',
                                    'julian lemos': 'julian lemos', 'leonardo gadelha': 'leonardo gadelha', 'alexandre silveira': 'alexandre silveira', 'tia eron': 'tia eron', 'jorielson': 'jorielson', 
                                   'vilalba': 'vilalba', 'patrick dorneles': 'patrick dorneles', 'gastão vieira': 'gastao vieira', 'roman': 'roman', 'reinhold stephanes junior': 'reinhold stephanes junior',
                                   'ted conti': 'ted conti', 'paulo vicente caleffi': 'paulo vicente caleffi', 'valdevan noventa': 'valdevan noventa', 'alencar santana braga': 'alencar santana braga', 
                                   'bia cavassa': 'bia cavassa', 'ely santos': 'ely santos', 'marcelo brum': 'marcelo brum', 'josé nunes': 'jose nunes', 'pedro dalua': 'pedro dalua', 
                                   'fernando coelho filho': 'fernando coelho filho', 'guilherme derrite': 'guilherme derrite', 'bruna furlan': 'bruna furlan', 'miguel haddad': 'miguel haddad', 
                                    'rossoni': 'rossoni', 'marcelo squassoni': 'marcelo squassoni', 'josé mentor': 'jose mentor', 'rose de freitas': 'rose de freitas', 
                                    'paulo pereira da silva': 'paulo pereira da silva', 'josé aníbal': 'jose anibal', 'aníbal gomes': 'anibal gomes', 'zé augusto nalin': 'ze augusto nalin',
                                    'leonardo picciani': 'leonardo picciani', 'henrique do paraíso': 'henrique do paraiso', 'dalua do rota': 'dalua do rota', 'boca aberta': 'boca aberta', 
                                    'odorico monteiro': 'odorico monteiro', 'totonho lopes': 'totonho lopes', 'rafafá': 'rafafa', 'laerte bessa': 'laerte bessa', 'flordelis': 'flordelis', 
                                    'dr gonçalo': 'dr goncalo', 'nilda gondim': 'nilda gondim', 'eduardo gomes': 'eduardo gomes', 'henrique afonso': 'henrique afonso', 'norma pereira': 'norma pereira', 
                                    'schiavinato': 'schiavinato', 'gorete pereira': 'gorete pereira', 'jorge bittar': 'jorge bittar', 'santini': 'santini', 'marcio bittar': 'marcio bittar', 
                                    'romário': 'romario', 'rachel marques': 'rachel marques', 'aureo': 'aureo', 'esperidião amin': 'esperidiao amin', 'marçal filho': 'marcal filho', 
                                    'dr geraldo resende': 'dr geraldo resende', 'coronel ulysses': 'coronel ulysses', 'dr eduardo velloso': 'dr eduardo velloso', 'gerlen diniz': 'gerlen diniz', 
                                    'meire serafim': 'meire serafim', 'roberto duarte': 'roberto duarte', 'socorro neri': 'socorro neri', 'zezinho barbary': 'zezinho barbary', 'alfredo gaspar': 'alfredo gaspar',
                                    'daniel barbosa': 'daniel barbosa', 'delegado fabio costa': 'delegado fabio costa', 'luciano amaral': 'luciano amaral', 'rafael brito (tio rafa)': 'rafael brito (tio rafa)',
                                    'adail filho': 'adail filho', 'amom mandel': 'amom mandel', 'fausto santos jr': 'fausto santos jr', 'saullo vianna': 'saullo vianna', 'dorinaldo malafaia': 'dorinaldo malafaia',
                                   'dr. pupio': 'dr pupio', 'josenildo': 'josenildo', 'professora goreth': 'professora goreth', 'silvia waiãpi': 'silvia waiapi', 'sonize barbosa': 'sonize barbosa', 
                                    'capitão alden': 'capitao alden', 'deputado dal': 'deputado dal', 'diego coronel': 'diego coronel', 'gabriel nunes': 'gabriel nunes', 'ivoneide caetano': 'ivoneide caetano',
                                    'joão leão': 'joao leao', 'léo prates': 'leo prates', 'neto carletto': 'neto carletto', 'ricardo maia': 'ricardo maia', 'roberta roma': 'roberta roma', 
                                    'rogeria santos': 'rogeria santos', 'andré fernandes': 'andre fernandes', 'dayany do capitão': 'dayany do capitao', 'eunício': 'eunicio', 'fernanda pessoa': 'fernanda pessoa', 
                                    'luiz gastão': 'luiz gastao', 'matheus noronha': 'matheus noronha', 'yury do paredão': 'yury do paredao', 'fraga': 'fraga', 'fred linhares': 'fred linhares', 
                                    'gilvan maximo': 'gilvan maximo', 'professor reginaldo veras': 'professor reginaldo veras', 'rafael prudente': 'rafael prudente', 'dr. victor': 'dr victor', 
                                    'gilson daniel': 'gilson daniel', 'gilvan o federal da direita': 'gilvan o federal da direita', 'jack rocha': 'jack rocha', 'messias donato': 'messias donato', 
                                    'daniel agrobom': 'daniel agrobom', 'delegada adriana accorsi': 'delegada adriana accorsi', 'dr ismael alexandrino': 'dr ismael alexandrino', 'gustavo gayer': 'gustavo gayer',
                                    'jeferson rodrigues': 'jeferson rodrigues', 'lêda borges': 'leda borges', 'marussa boldrin': 'marussa boldrin', 'silvye alves': 'silvye alves', 'amanda gentil': 'amanda gentil', 
                                    'detinha': 'detinha', 'duarte': 'duarte', 'fábio macedo': 'fabio macedo', 'márcio honaiser': 'marcio honaiser', 'roseana sarney': 'roseana sarney', 
                                    'ana paula junqueira leao': 'ana paula junqueira leao', 'ana pimentel': 'ana pimentel', 'bruno farias': 'bruno farias', 'célia xakriabá': 'celia xakriaba', 
                                    'dandara': 'dandara', 'delegada ione barbosa': 'delegada ione barbosa', 'duda salabert': 'duda salabert', 'luiz fernando': 'luiz fernando', 
                                    'mauricio do volei': 'mauricio do volei', 'miguel ângelo': 'miguel angelo', 'nely aquino': 'nely aquino', 'nikolas ferreira': 'nikolas ferreira', 
                                    'pedro aihara': 'pedro aihara', 'rafael simoes': 'rafael simoes', 'rosângela reis': 'rosangela reis', 'samuel viana': 'samuel viana', 'camila jara': 'camila jara', 
                                    'marcos pollon': 'marcos pollon', 'rodolfo nogueira': 'rodolfo nogueira', 'abílio': 'abilio', 'amália barros': 'amalia barros', 'coronel assis': 'coronel assis', 
                                    'coronel fernanda': 'coronel fernanda', 'fábio garcia': 'fabio garcia', 'andreia siqueira': 'andreia siqueira', 'antonio doido': 'antonio doido', 
                                    'delegado caveira': 'delegado caveira', 'dilvanda faro': 'dilvanda faro', 'dra alessandra haber': 'dra alessandra haber', 'henderson pinto': 'henderson pinto', 
                                    'keniston': 'keniston', 'raimundo santos': 'raimundo santos', 'renilce nicodemos': 'renilce nicodemos', 'cabo gilberto silva': 'cabo gilberto silva', 
                                    'luiz couto': 'luiz couto', 'mersinho lucena': 'mersinho lucena', 'murilo galdino': 'murilo galdino', 'romero rodrigues': 'romero rodrigues', 
                                    'clarissa tércio': 'clarissa tercio', 'clodoaldo magalhães': 'clodoaldo magalhaes', 'coronel meira': 'coronel meira', 'eriberto medeiros': 'eriberto medeiros',
                                    'guilherme uchoa junior': 'guilherme uchoa junior', 'iza arruda': 'iza arruda', 'lucas ramos': 'lucas ramos', 'lula da fonte': 'lula da fonte',
                                    'maria arraes': 'maria arraes', 'mendonça filho': 'mendonca filho', 'pedro campos': 'pedro campos', 'waldemar oliveira': 'waldemar oliveira', 'atila': 'atila', 
                                    'castro neto': 'castro neto', 'dr. francisco': 'dr francisco', 'florentino neto': 'florentino neto', 'jadyel da jupi': 'jadyel da jupi', 
                                    'julio arcoverde': 'julio arcoverde', 'beto preto': 'beto preto', 'beto richa': 'beto richa', 'carol dartora': 'carol dartora', 
                                    'delegado matheus laiola': 'delegado matheus laiola', 'deltan dallagnol': 'deltan dallagnol', 'dilceu sperafico': 'dilceu sperafico', 
                                    'geraldo mendes': 'geraldo mendes', 'padovani': 'padovani', 'paulo litro': 'paulo litro', 'tadeu veneri': 'tadeu veneri', 
                                    'tião medeiros': 'tiao medeiros', 'bandeira de mello': 'bandeira de mello', 'chico alencar': 'chico alencar', 'dani cunha': 'dani cunha',
                                    'daniel soranz': 'daniel soranz', 'delegado ramagem': 'delegado ramagem', 'dimas gadelha': 'dimas gadelha', 'general pazuello': 'general pazuello',
                                    'lindbergh farias': 'lindbergh farias', 'luciano vieira': 'luciano vieira', 'marcelo crivella': 'marcelo crivella', 'marcelo queiroz': 'marcelo queiroz', 
                                    'marcos tavares': 'marcos tavares', 'max': 'max', 'murillo gouvea': 'murillo gouvea', 'pastor henrique vieira': 'pastor henrique vieira', 'reimont': 'reimont', 
                                   'roberto monteiro pai': 'roberto monteiro pai', 'sargento portugal': 'sargento portugal', 'tarcísio motta': 'tarcisio motta', 'washington quaquá': 'washington quaqua', 
                                   'mineiro': 'mineiro', 'paulinho freire': 'paulinho freire', 'robinson faria': 'robinson faria', 'sargento gonçalves': 'sargento goncalves', 'cristiane lopes': 'cristiane lopes',
                                   'dr fernando máximo': 'dr fernando maximo', 'lebrão': 'lebrao', 'maurício carvalho': 'mauricio carvalho', 'thiago flores': 'thiago flores', 'albuquerque': 'albuquerque', 
                                    'defensor stélio dener': 'defensor stelio dener', 'duda ramos': 'duda ramos', 'helena da asatur': 'helena da asatur', 'pastor diniz': 'pastor diniz', 
                                   'zé haroldo cathedral': 'ze haroldo cathedral', 'alcibio mesquita bibo nunes': 'alcibio mesquita bibo nunes', 'alexandre lindenmeyer': 'alexandre lindenmeyer', 
                                   'any ortiz': 'any ortiz', 'busato': 'busato', 'daiana santos': 'daiana santos', 'denise pessôa': 'denise pessoa', 'franciane bayer': 'franciane bayer', 
                                   'mauricio marcon': 'mauricio marcon', 'tenente coronel zucco': 'tenente coronel zucco', 'ana paula lima': 'ana paula lima', 'cobalchini': 'cobalchini', 
                                    'daniela reinehr': 'daniela reinehr', 'ismael': 'ismael', 'julia zanatta': 'julia zanatta', 'rafael pezenti': 'rafael pezenti', 'zé trovão': 'ze trovao', 
                                   'delegada katarina': 'delegada katarina', 'icaro de valmir': 'icaro de valmir', 'rodrigo valadares': 'rodrigo valadares', 'thiago de joaldo': 'thiago de joaldo', 
                                    'yandra de andré': 'yandra de andre', 'alberto mourão': 'alberto mourao', 'antonio carlos rodrigues': 'antonio carlos rodrigues', 'bruno ganem': 'bruno ganem', 
                                    'delegado bruno lima': 'delegado bruno lima', 'delegado da cunha': 'delegado da cunha', 'delegado palumbo': 'delegado palumbo', 'delegado paulo bilynskyj': 'delegado paulo bilynskyj',
                                   'erika hilton': 'erika hilton', 'fabio teruel': 'fabio teruel', 'felipe becari': 'felipe becari', 'guilherme boulos': 'guilherme boulos', 'jilmar tatto': 'jilmar tatto', 
                                    'jonas donizette': 'jonas donizette', 'juliana cardoso': 'juliana cardoso', 'kiko celeguim': 'kiko celeguim', 'luiz marinho': 'luiz marinho', 'marangoni': 'marangoni',
                                    'marcelo lima': 'marcelo lima', 'marina silva': 'marina silva', 'mario frias': 'mario frias', 'mauricio neves': 'mauricio neves', 'paulo alexandre barbosa': 'paulo alexandre barbosa',
                                    'ricardo salles': 'ricardo salles', 'rodrigo gambale': 'rodrigo gambale', 'rosângela moro': 'rosangela moro', 'simone marquetto': 'simone marquetto', 
                                    'sônia guajajara': 'sonia guajajara', 'alexandre guimarães': 'alexandre guimaraes', 'filipe martins': 'filipe martins', 'ricardo ayres': 'ricardo ayres', 
                                    'Bispo José': 'bispo jose', 'Marcio Bittar': 'marcio bittar', 'Sérgio Petecão': 'sergio petecao', 'Fernando Collor': 'fernando collor', 'Renan Calheiros': 'renan calheiros', 'Rodrigo Cunha': 'rodrigo cunha', 'Eduardo Braga': 'eduardo braga', 'Omar Aziz': 'omar aziz', 'Plínio Valério': 'plinio valerio', 'Davi Alcolumbre': 'davi alcolumbre', 'Lucas Barreto': 'lucas barreto', 'Randolfe Rodrigues': 'randolfe rodrigues', 'Angelo Coronel': 'angelo coronel', 'Jaques Wagner': 'jaques wagner', 'Otto Alencar': 'otto alencar', 'Cid Gomes': 'cid gomes', 'Eduardo Girão': 'eduardo girao', 'Tasso Jereissati': 'tasso jereissati', 'Izalci Lucas': 'izalci lucas', 'Leila Barros': 'leila barros', 'Reguffe': 'reguffe', 'Fabiano Contarato': 'fabiano contarato', 'Marcos do Val': 'marcos do val', 'Rose de Freitas': 'rose de freitas', 'Jorge Kajuru': 'jorge kajuru', 'Luiz Carlos do Carmo': 'luiz carlos do carmo', 'Vanderlan Cardoso': 'vanderlan cardoso', 'Eliziane Gama': 'eliziane gama', 'Roberto Rocha': 'roberto rocha', 'Weverton': 'weverton', 'Carlos Viana': 'carlos viana', 'Rodrigo Pacheco': 'rodrigo pacheco', 'Nelsinho Trad': 'nelsinho trad', 'Soraya Thronicke': 'soraya thronicke', 'Jayme Campos': 'jayme campos', 'Margareth Buzetti': 'margareth buzetti', 'Wellington Fagundes': 'wellington fagundes', 'Jader Barbalho': 'jader barbalho', 'Paulo Rocha': 'paulo rocha', 'Zequinha Marinho': 'zequinha marinho', 'Daniella Ribeiro': 'daniella ribeiro', 'Nilda Gondim': 'nilda gondim', 'Veneziano Vital do Rêgo': 'veneziano vital do rego', 'Fernando Bezerra Coelho': 'fernando bezerra coelho', 'Fernando Dueire': 'fernando dueire', 'Humberto Costa': 'humberto costa', 'Ciro Nogueira': 'ciro nogueira', 'Elmano Férrer': 'elmano ferrer', 'Marcelo Castro': 'marcelo castro', 'Alvaro Dias': 'alvaro dias', 'Flávio Arns': 'flavio arns', 'Oriovisto Guimarães': 'oriovisto guimaraes', 'Carlos Portinho': 'carlos portinho', 'Flávio Bolsonaro': 'flavio bolsonaro', 'Romário': 'romario', 'Jean Paul Prates': 'jean paul prates', 'Styvenson Valentim': 'styvenson valentim', 'Zenaide Maia': 'zenaide maia', 'Acir Gurgacz': 'acir gurgacz', 'Confúcio Moura': 'confucio moura', 'Dr. Samuel Araújo': 'dr. samuel araujo', 'Chico Rodrigues': 'chico rodrigues', 'Mecias de Jesus': 'mecias de jesus', 'Telmário Mota': 'telmario mota', 'Lasier Martins': 'lasier martins', 'Luis Carlos Heinze': 'luis carlos heinze', 'Paulo Paim': 'paulo paim', 'Dário Berger': 'dario berger', 'Esperidião Amin': 'esperidiao amin', 'Ivete da Silveira': 'ivete da silveira', 'Alessandro Vieira': 'alessandro vieira', 'Maria do Carmo Alves': 'maria do carmo alves', 'Rogério Carvalho': 'rogerio carvalho', 'Giordano': 'giordano', 'José Serra': 'jose serra', 'Mara Gabrilli': 'mara gabrilli', 'Eduardo Gomes': 'eduardo gomes', 'Irajá': 'iraja', 'Kátia Abreu': 'katia abreu'})
proposicoes2

,id_proposicao,id_deputado,politico,partido,uf,ano,tipo,ementa,palavras_chave,status,data,estado,região
0,2343468,204457,juninho do pneu,UNIÃO,RJ,2019,Parecer do Relator,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",NaN,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",2019-07-03,Rio de Janeiro,Sudeste
1,2237302,141450,hugo leal,PSD,RJ,2019,Emenda ao Substitutivo,Suprima-se a art. 115 do Substitutivo ao Proje...,NaN,Apresentação da Emenda ao Substitutivo n. 84 P...,2019-12-10,Rio de Janeiro,Sudeste
2,2237301,141450,hugo leal,PSD,RJ,2019,Emenda ao Substitutivo,Acrescenta-se o Parágrafo único ao art. 64 con...,NaN,Apresentação da Emenda ao Substitutivo n. 83 P...,2019-12-10,Rio de Janeiro,Sudeste
3,2237300,141450,hugo leal,PSD,RJ,2019,Emenda ao Substitutivo,Acrescenta-se o art. 101 constante do art. 1º ...,NaN,Apresentação da Emenda ao Substitutivo n. 82 P...,2019-12-10,Rio de Janeiro,Sudeste
4,2237259,141450,hugo leal,PSD,RJ,2019,Emenda ao Substitutivo,"Altera a Lei nº 9.503, de 23 de setembro de 19...",NaN,Apresentação da Emenda ao Substitutivo n. 29 P...,2019-12-04,Rio de Janeiro,Sudeste
...,...,...,...,...,...,...,...,...,...,...,...,...,...
276154,2268473,204449,nelson barbudo,PSL,MT,2021,Adesão a Bloco Parlamentar,Assunto: Adesão a Bloco Parlamentar.,NaN,Relatório de Conferência de Assinaturas Eletrô...,2021-01-19,Mato Grosso,Centro-Oeste
276155,2268467,198783,emanuel pinheiro neto,PTB,MT,2021,Adesão a Bloco Parlamentar,adesão do Partido Trabalhista Brasileiro – PTB...,NaN,Relatório de Conferência de Assinaturas Eletrô...,2021-01-19,Mato Grosso,Centro-Oeste
276156,2268428,204449,nelson barbudo,PSL,MT,2021,Adesão a Bloco Parlamentar,Adesão a Bloco Parlamentar,NaN,Apresentação do Adesão a Bloco Parlamentar n. ...,2021-01-07,Mato Grosso,Centro-Oeste
276157,2268427,146307,neri geller,PP,MT,2021,Constituição de Bloco Parlamentar,Constitui Bloco Parlamentar.,NaN,Apresentação do Constituição de Bloco Parlamen...,2021-01-07,Mato Grosso,Centro-Oeste


## Caixa baixa e limpeza
---



In [ ]:
# Letras em caixa baixa
proposicoes2['ementa_limpa'] = proposicoes2['ementa'].str.lower().map(lambda x: re.sub(r'[^\w\s]', '', x))
proposicoes2

,id_proposicao,id_deputado,politico,partido,uf,ano,tipo,ementa,palavras_chave,status,data,estado,região,ementa_limpa
0,2343468,204457,juninho do pneu,UNIÃO,RJ,2019,Parecer do Relator,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",NaN,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",2019-07-03,Rio de Janeiro,Sudeste,parecer do relator dep juninho do pneu demrj p...
1,2237302,141450,hugo leal,PSD,RJ,2019,Emenda ao Substitutivo,Suprima-se a art. 115 do Substitutivo ao Proje...,NaN,Apresentação da Emenda ao Substitutivo n. 84 P...,2019-12-10,Rio de Janeiro,Sudeste,suprimase a art 115 do substitutivo ao projeto...
2,2237301,141450,hugo leal,PSD,RJ,2019,Emenda ao Substitutivo,Acrescenta-se o Parágrafo único ao art. 64 con...,NaN,Apresentação da Emenda ao Substitutivo n. 83 P...,2019-12-10,Rio de Janeiro,Sudeste,acrescentase o parágrafo único ao art 64 const...
3,2237300,141450,hugo leal,PSD,RJ,2019,Emenda ao Substitutivo,Acrescenta-se o art. 101 constante do art. 1º ...,NaN,Apresentação da Emenda ao Substitutivo n. 82 P...,2019-12-10,Rio de Janeiro,Sudeste,acrescentase o art 101 constante do art 1º do ...
4,2237259,141450,hugo leal,PSD,RJ,2019,Emenda ao Substitutivo,"Altera a Lei nº 9.503, de 23 de setembro de 19...",NaN,Apresentação da Emenda ao Substitutivo n. 29 P...,2019-12-04,Rio de Janeiro,Sudeste,altera a lei nº 9503 de 23 de setembro de 1997...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276154,2268473,204449,nelson barbudo,PSL,MT,2021,Adesão a Bloco Parlamentar,Assunto: Adesão a Bloco Parlamentar.,NaN,Relatório de Conferência de Assinaturas Eletrô...,2021-01-19,Mato Grosso,Centro-Oeste,assunto adesão a bloco parlamentar
276155,2268467,198783,emanuel pinheiro neto,PTB,MT,2021,Adesão a Bloco Parlamentar,adesão do Partido Trabalhista Brasileiro – PTB...,NaN,Relatório de Conferência de Assinaturas Eletrô...,2021-01-19,Mato Grosso,Centro-Oeste,adesão do partido trabalhista brasileiro ptb ...
276156,2268428,204449,nelson barbudo,PSL,MT,2021,Adesão a Bloco Parlamentar,Adesão a Bloco Parlamentar,NaN,Apresentação do Adesão a Bloco Parlamentar n. ...,2021-01-07,Mato Grosso,Centro-Oeste,adesão a bloco parlamentar
276157,2268427,146307,neri geller,PP,MT,2021,Constituição de Bloco Parlamentar,Constitui Bloco Parlamentar.,NaN,Apresentação do Constituição de Bloco Parlamen...,2021-01-07,Mato Grosso,Centro-Oeste,constitui bloco parlamentar


In [ ]:
proposicoes2['ementa_limpa'] = proposicoes2['ementa_limpa'].str.replace('\d+', '')
proposicoes2

<ipython-input-31-441f2e3ceebf>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  proposicoes2['ementa_limpa'] = proposicoes2['ementa_limpa'].str.replace('\d+', '')


,id_proposicao,id_deputado,politico,partido,uf,ano,tipo,ementa,palavras_chave,status,data,estado,região,ementa_limpa
0,2343468,204457,juninho do pneu,UNIÃO,RJ,2019,Parecer do Relator,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",NaN,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",2019-07-03,Rio de Janeiro,Sudeste,parecer do relator dep juninho do pneu demrj p...
1,2237302,141450,hugo leal,PSD,RJ,2019,Emenda ao Substitutivo,Suprima-se a art. 115 do Substitutivo ao Proje...,NaN,Apresentação da Emenda ao Substitutivo n. 84 P...,2019-12-10,Rio de Janeiro,Sudeste,suprimase a art do substitutivo ao projeto de...
2,2237301,141450,hugo leal,PSD,RJ,2019,Emenda ao Substitutivo,Acrescenta-se o Parágrafo único ao art. 64 con...,NaN,Apresentação da Emenda ao Substitutivo n. 83 P...,2019-12-10,Rio de Janeiro,Sudeste,acrescentase o parágrafo único ao art constan...
3,2237300,141450,hugo leal,PSD,RJ,2019,Emenda ao Substitutivo,Acrescenta-se o art. 101 constante do art. 1º ...,NaN,Apresentação da Emenda ao Substitutivo n. 82 P...,2019-12-10,Rio de Janeiro,Sudeste,acrescentase o art constante do art º do subs...
4,2237259,141450,hugo leal,PSD,RJ,2019,Emenda ao Substitutivo,"Altera a Lei nº 9.503, de 23 de setembro de 19...",NaN,Apresentação da Emenda ao Substitutivo n. 29 P...,2019-12-04,Rio de Janeiro,Sudeste,altera a lei nº de de setembro de que insti...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276154,2268473,204449,nelson barbudo,PSL,MT,2021,Adesão a Bloco Parlamentar,Assunto: Adesão a Bloco Parlamentar.,NaN,Relatório de Conferência de Assinaturas Eletrô...,2021-01-19,Mato Grosso,Centro-Oeste,assunto adesão a bloco parlamentar
276155,2268467,198783,emanuel pinheiro neto,PTB,MT,2021,Adesão a Bloco Parlamentar,adesão do Partido Trabalhista Brasileiro – PTB...,NaN,Relatório de Conferência de Assinaturas Eletrô...,2021-01-19,Mato Grosso,Centro-Oeste,adesão do partido trabalhista brasileiro ptb ...
276156,2268428,204449,nelson barbudo,PSL,MT,2021,Adesão a Bloco Parlamentar,Adesão a Bloco Parlamentar,NaN,Apresentação do Adesão a Bloco Parlamentar n. ...,2021-01-07,Mato Grosso,Centro-Oeste,adesão a bloco parlamentar
276157,2268427,146307,neri geller,PP,MT,2021,Constituição de Bloco Parlamentar,Constitui Bloco Parlamentar.,NaN,Apresentação do Constituição de Bloco Parlamen...,2021-01-07,Mato Grosso,Centro-Oeste,constitui bloco parlamentar


In [ ]:
proposicoes = pd.merge(proposicoes2, reeleitos, on='politico')
proposicoes

,id_proposicao,id_deputado,politico,partido,uf,ano,tipo,ementa,palavras_chave,status,data,estado,região,ementa_limpa,eleição
0,2343468,204457,juninho do pneu,UNIÃO,RJ,2019,Parecer do Relator,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",NaN,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",2019-07-03,Rio de Janeiro,Sudeste,parecer do relator dep juninho do pneu demrj p...,Reeleito(a)
1,2235983,204457,juninho do pneu,DEM,RJ,2019,Substitutivo,Dispõe sobre a construção de bicicletários em ...,NaN,"Apresentação do Substitutivo n. 4 CVT, pelo De...",2019-12-19,Rio de Janeiro,Sudeste,dispõe sobre a construção de bicicletários em ...,Reeleito(a)
2,2235970,204457,juninho do pneu,DEM,RJ,2019,Parecer do Relator,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",NaN,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",2019-12-19,Rio de Janeiro,Sudeste,parecer do relator dep juninho do pneu demrj p...,Reeleito(a)
3,2234684,204457,juninho do pneu,DEM,RJ,2019,Projeto de Lei,Dispõe sobre o exercício da profissão de técni...,"Regulamentação profissional, técnico em necrop...","Recebimento pela CSSF, apensado ao PL-10674/2018",2022-11-18,Rio de Janeiro,Sudeste,dispõe sobre o exercício da profissão de técni...,Reeleito(a)
4,2234681,204457,juninho do pneu,DEM,RJ,2019,Projeto de Lei,Institui o programa médico nas farmácias.,"Criação, Programa médico nas farmácias, permis...",Encerrado o prazo de 5 sessões para apresentaç...,2021-10-27,Rio de Janeiro,Sudeste,institui o programa médico nas farmácias,Reeleito(a)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202504,2283425,141552,valtenir pereira,MDB,MT,2021,Requerimento de Inclusão de Matéria na Ordem d...,"Requer nos termos do art. 114, inciso XIV, do ...",NaN,Publicação inicial no DCD do dia 11/03/2022,2022-03-10,Mato Grosso,Centro-Oeste,requer nos termos do art inciso xiv do ricd a...,Não reeleito
202505,2261365,NaN,jayme campos,DEM,MT,2020,Emenda na Comissão,Dá nova redação à MPV 996/2020,NaN,"Apresentação da Emenda na Comissão, EMC 35/202...",2020-08-28,Mato Grosso,Centro-Oeste,dá nova redação à mpv,Senador
202506,2261364,NaN,jayme campos,DEM,MT,2020,Emenda na Comissão,Dá nova redação à MPV 996/2020,NaN,"Apresentação da Emenda na Comissão, EMC 34/202...",2020-08-28,Mato Grosso,Centro-Oeste,dá nova redação à mpv,Senador
202507,2316526,NaN,jayme campos,DEM,MT,2022,Emenda na Comissão,Dá nova redação à MPV 1099/2022,NaN,"Apresentação da Emenda na Comissão, EMC 209/20...",2022-02-03,Mato Grosso,Centro-Oeste,dá nova redação à mpv,Senador


## Stop words

In [ ]:
stop = stopwords.words('portuguese') + ['d','de','do','e','a','da','o','para','nº','que','requer','ao','em','no','dos','à','altera','pela','art.','os','das','com','pl','parecer','na','dá','projeto','dep.',
                                        'lei','sobre','-','relator,','realização','dispõe','comissão','federal','emenda','redação','nova','disposições','públicos','mpv','outras','ministro','audiência','organização',
                                        'institui','termos','empregados','–','urgência','dezembro','servidores,','aprovação.','informações','pec','inclusão','requerimento','2021,','acrescenta','federal,','contra',
                                        'apreciação','dia','programa','dispor','inciso','artigo','debater','2021.','§','aprovação','sugere','ato','ministério','pessoas','julho','sr.','arts.','plenário','solicita',
                                        'deputados,','âmbito','município','frente','interno','coautoria','decreto','junho','técnica','dispositivos','1º','n°','sistema','serviços','marcelo','setembro','substitutivo',
                                        'sessão','emendas','agentes','criação','direitos','janeiro','março','registro','efeitos','proposta','senhor','deputados','recursos','n.','constitucionalidade,','constitucionais',
                                        'art', 'º', 'ª', 'dep', 'relator', 'n', 'voto', 'deste', 'projeto','lei','dispoe','sobre','relator','altera','requer','realizacao','servidores','regimento','câmara','constitucionalidade',
                                        'juridicidade','ramos','plam','nacional','cargo','função','mista', 'interstício','executivo','regimental','dispensa', 'bloco','parlamentar','adição',
                                        'coautorias','trata','demonstração','empenhos','rio','grande','segundo','turno','deliberação','arts','impacta','afeta','of','estados','distrito','solene',
                                        'homenagem','transitórias','regras','b','c','psgse','vossa','excelência', 'f', 'g']

In [ ]:
proposicoes['ementa_limpa'] = proposicoes['ementa_limpa'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
proposicoes.head()

,id_proposicao,id_deputado,politico,partido,uf,ano,tipo,ementa,palavras_chave,status,data,estado,região,ementa_limpa,eleição
0,2343468,204457,juninho do pneu,UNIÃO,RJ,2019,Parecer do Relator,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",NaN,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",2019-07-03,Rio de Janeiro,Sudeste,juninho pneu demrj,Reeleito(a)
1,2235983,204457,juninho do pneu,DEM,RJ,2019,Substitutivo,Dispõe sobre a construção de bicicletários em ...,NaN,"Apresentação do Substitutivo n. 4 CVT, pelo De...",2019-12-19,Rio de Janeiro,Sudeste,construção bicicletários órgãos federais,Reeleito(a)
2,2235970,204457,juninho do pneu,DEM,RJ,2019,Parecer do Relator,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",NaN,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",2019-12-19,Rio de Janeiro,Sudeste,juninho pneu demrj apensados,Reeleito(a)
3,2234684,204457,juninho do pneu,DEM,RJ,2019,Projeto de Lei,Dispõe sobre o exercício da profissão de técni...,"Regulamentação profissional, técnico em necrop...","Recebimento pela CSSF, apensado ao PL-10674/2018",2022-11-18,Rio de Janeiro,Sudeste,exercício profissão técnico necropsia,Reeleito(a)
4,2234681,204457,juninho do pneu,DEM,RJ,2019,Projeto de Lei,Institui o programa médico nas farmácias.,"Criação, Programa médico nas farmácias, permis...",Encerrado o prazo de 5 sessões para apresentaç...,2021-10-27,Rio de Janeiro,Sudeste,médico farmácias,Reeleito(a)


### Filtrar palavras para *stopword*

In [ ]:
lista_palavras = proposicoes.ementa_limpa.str.split(expand=True).stack().value_counts()
lista_palavras

constituição    35747
providências    24454
social          21156
pública         17646
previdência     15379
                ...  
itb                 1
dinoá               1
codern              1
tomazoni            1
equiparandoo        1
Length: 30045, dtype: int64

## Aplicar n-grams

In [ ]:
n = 2
proposicoes['ngrams'] = proposicoes['ementa_limpa'].str.split().apply(lambda x: list(map(' '.join, ngrams(x, n=n))))
prop = (proposicoes.assign(count=proposicoes['ngrams'].str.len()).explode('ngrams').query('count > 0'))
prop.head()

,id_proposicao,id_deputado,politico,partido,uf,ano,tipo,ementa,palavras_chave,status,data,estado,região,ementa_limpa,eleição,ngrams,count
0,2343468,204457,juninho do pneu,UNIÃO,RJ,2019,Parecer do Relator,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",NaN,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",2019-07-03,Rio de Janeiro,Sudeste,juninho pneu demrj,Reeleito(a),juninho pneu,2
0,2343468,204457,juninho do pneu,UNIÃO,RJ,2019,Parecer do Relator,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",NaN,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",2019-07-03,Rio de Janeiro,Sudeste,juninho pneu demrj,Reeleito(a),pneu demrj,2
1,2235983,204457,juninho do pneu,DEM,RJ,2019,Substitutivo,Dispõe sobre a construção de bicicletários em ...,NaN,"Apresentação do Substitutivo n. 4 CVT, pelo De...",2019-12-19,Rio de Janeiro,Sudeste,construção bicicletários órgãos federais,Reeleito(a),construção bicicletários,3
1,2235983,204457,juninho do pneu,DEM,RJ,2019,Substitutivo,Dispõe sobre a construção de bicicletários em ...,NaN,"Apresentação do Substitutivo n. 4 CVT, pelo De...",2019-12-19,Rio de Janeiro,Sudeste,construção bicicletários órgãos federais,Reeleito(a),bicicletários órgãos,3
1,2235983,204457,juninho do pneu,DEM,RJ,2019,Substitutivo,Dispõe sobre a construção de bicicletários em ...,NaN,"Apresentação do Substitutivo n. 4 CVT, pelo De...",2019-12-19,Rio de Janeiro,Sudeste,construção bicicletários órgãos federais,Reeleito(a),órgãos federais,3


In [ ]:
prop = prop.drop(columns=['count'])
prop

,id_proposicao,id_deputado,politico,partido,uf,ano,tipo,ementa,palavras_chave,status,data,estado,região,ementa_limpa,eleição,ngrams
0,2343468,204457,juninho do pneu,UNIÃO,RJ,2019,Parecer do Relator,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",NaN,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",2019-07-03,Rio de Janeiro,Sudeste,juninho pneu demrj,Reeleito(a),juninho pneu
0,2343468,204457,juninho do pneu,UNIÃO,RJ,2019,Parecer do Relator,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",NaN,"Parecer do Relator, Dep. Juninho do Pneu (DEM-...",2019-07-03,Rio de Janeiro,Sudeste,juninho pneu demrj,Reeleito(a),pneu demrj
1,2235983,204457,juninho do pneu,DEM,RJ,2019,Substitutivo,Dispõe sobre a construção de bicicletários em ...,NaN,"Apresentação do Substitutivo n. 4 CVT, pelo De...",2019-12-19,Rio de Janeiro,Sudeste,construção bicicletários órgãos federais,Reeleito(a),construção bicicletários
1,2235983,204457,juninho do pneu,DEM,RJ,2019,Substitutivo,Dispõe sobre a construção de bicicletários em ...,NaN,"Apresentação do Substitutivo n. 4 CVT, pelo De...",2019-12-19,Rio de Janeiro,Sudeste,construção bicicletários órgãos federais,Reeleito(a),bicicletários órgãos
1,2235983,204457,juninho do pneu,DEM,RJ,2019,Substitutivo,Dispõe sobre a construção de bicicletários em ...,NaN,"Apresentação do Substitutivo n. 4 CVT, pelo De...",2019-12-19,Rio de Janeiro,Sudeste,construção bicicletários órgãos federais,Reeleito(a),órgãos federais
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202503,2284029,141552,valtenir pereira,MDB,MT,2021,Requerimento de Informação,Requer ao Ministro de Estado de Minas e Energi...,NaN,Encaminhamento de resposta por meio do sistema...,2021-07-06,Mato Grosso,Centro-Oeste,estado minas energia sr bento albuquerque escl...,Não reeleito,direito prioridade
202503,2284029,141552,valtenir pereira,MDB,MT,2021,Requerimento de Informação,Requer ao Ministro de Estado de Minas e Energi...,NaN,Encaminhamento de resposta por meio do sistema...,2021-07-06,Mato Grosso,Centro-Oeste,estado minas energia sr bento albuquerque escl...,Não reeleito,prioridade plg
202504,2283425,141552,valtenir pereira,MDB,MT,2021,Requerimento de Inclusão de Matéria na Ordem d...,"Requer nos termos do art. 114, inciso XIV, do ...",NaN,Publicação inicial no DCD do dia 11/03/2022,2022-03-10,Mato Grosso,Centro-Oeste,xiv ricd ordem constituição,Não reeleito,xiv ricd
202504,2283425,141552,valtenir pereira,MDB,MT,2021,Requerimento de Inclusão de Matéria na Ordem d...,"Requer nos termos do art. 114, inciso XIV, do ...",NaN,Publicação inicial no DCD do dia 11/03/2022,2022-03-10,Mato Grosso,Centro-Oeste,xiv ricd ordem constituição,Não reeleito,ricd ordem


In [ ]:
prop = prop.drop_duplicates()
prop.shape[0]

1712352

In [ ]:
ngrams_total = prop['ngrams'].value_counts()
ngrams_total

previdência social        13214
estabelece transição      11115
social estabelece         10985
transição providências    10869
modifica previdência      10704
                          ...  
benefício segurado            1
solicitação benefício         1
direito solicitação           1
ambas garantir                1
grosso direito                1
Name: ngrams, Length: 224906, dtype: int64

In [ ]:
df_prop = pd.DataFrame(ngrams_total)
df_prop = df_prop.reset_index(level=0)
df_prop.columns = ['termos', 'total']
df_prop.head(60)

,termos,total
0,previdência social,13214
1,estabelece transição,11115
2,social estabelece,10985
3,transição providências,10869
4,modifica previdência,10704
5,marcos pereira,3548
6,tributário providências,3400
7,seguridade social,2576
8,pereira republicsp,2476
9,pandemia covid,2271


In [ ]:
selecao_prop = df_prop['total'] >= 200
termos_prop = df_prop[selecao_prop]
termos_prop

,termos,total
0,previdência social,13214
1,estabelece transição,11115
2,social estabelece,10985
3,transição providências,10869
4,modifica previdência,10704
...,...,...
891,educação profissional,200
892,educação sr,200
893,fazenda pública,200
894,empresas pequeno,200


In [ ]:
selecao1 = prop[prop['ngrams'].isin(['liberdade expressão', 'liberdade imprensa', 'acesso informação', 'banco dados', 'mercado câmbio', 'crédito consignado', 'bolsa familia',
                                     'legal startups', 'preço combustíveis', 'liberdade econômica', 'incentivo fiscal', 'defesa empreendedor', 'dados abertos', 'auxílio emergencial',
                                     'segurança pública', 'meio ambiente', 'seguridade social', 'agricultura pecuária', 'ciência tecnologia', 'direitos humanos', 'reforma previdência',
                                     'relações exteriores', 'defesa mulher', 'liberdade política', 'reforma administrativa', 'relações exteriores', 'defesa mulher', 'liberdade política',
                                     'contribuição social', 'financeira orçamentária', 'orçamentária financeira', 'teto gasto', 'teto gastos', 'responsabilidade fiscal', 'único saúde', 
                                     'auxílio brasil', 'povos indígenas'])]
selecao1.drop_duplicates()

,id_proposicao,id_deputado,politico,partido,uf,ano,tipo,ementa,palavras_chave,status,data,estado,região,ementa_limpa,eleição,ngrams
28,2222074,204457,juninho do pneu,DEM,RJ,2019,Emenda na Comissão,Altera o art. 195 para permitir a criação de c...,NaN,Relatório de Conferência de Assinaturas da EMC...,2019-09-26,Rio de Janeiro,Sudeste,permitir contribuição social rendimentos títul...,Reeleito(a),contribuição social
117,2205885,204457,juninho do pneu,DEM,RJ,2019,Emenda na Comissão,Adicione-se ao texto da Proposta de Emenda à C...,NaN,Relatório de Conferência de Assinaturas da EMC...,2019-05-30,Rio de Janeiro,Sudeste,adicionese texto constituição seguintes iia es...,Reeleito(a),contribuição social
135,2202176,204457,juninho do pneu,DEM,RJ,2019,Emenda na Comissão,Modifica a Proposta de Emenda à Constituição 6...,NaN,Relatório de Conferência de Assinaturas da EMC...,2019-05-28,Rio de Janeiro,Sudeste,modifica constituição previdência seguridade s...,Reeleito(a),seguridade social
135,2202176,204457,juninho do pneu,DEM,RJ,2019,Emenda na Comissão,Modifica a Proposta de Emenda à Constituição 6...,NaN,Relatório de Conferência de Assinaturas da EMC...,2019-05-28,Rio de Janeiro,Sudeste,modifica constituição previdência seguridade s...,Reeleito(a),segurança pública
178,2266020,204457,juninho do pneu,DEM,RJ,2020,Projeto de Lei,Dispõe sobre as diretrizes para o tratamento d...,"Diretrizes, procedimento, tratamento médico, d...","Recebimento pela CPD, apensado ao PL-4345/2016",2021-11-12,Rio de Janeiro,Sudeste,diretrizes tratamento doenças raras único saúde,Reeleito(a),único saúde
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202404,2200893,141552,valtenir pereira,MDB,MT,2019,Emenda na Comissão,Altera os arts. 40 e 201 da Constituição Feder...,NaN,Relatório de Conferência de Assinaturas da EMC...,2019-05-29,Mato Grosso,Centro-Oeste,constituição constituição seguridade social es...,Não reeleito,seguridade social
202417,2200218,141552,valtenir pereira,MDB,MT,2019,Emenda na Comissão,Altera as regras do financiamento da seguridad...,NaN,Relatório de Conferência de Assinaturas da EMC...,2019-05-08,Mato Grosso,Centro-Oeste,financiamento seguridade social meio incorpora...,Não reeleito,seguridade social
202418,2200093,141552,valtenir pereira,MDB,MT,2019,Parecer do Relator,"Parecer do Relator, Dep. Valtenir Pereira (MDB...",NaN,"Parecer do Relator, Dep. Valtenir Pereira (MDB...",2019-05-07,Mato Grosso,Centro-Oeste,valtenir pereira mdbmt regimentalidade legisla...,Não reeleito,ciência tecnologia
202424,2199570,141552,valtenir pereira,MDB,MT,2019,Parecer do Relator,"Parecer do Relator, Dep. Valtenir Pereira (MDB...",NaN,"Parecer do Relator, Dep. Valtenir Pereira (MDB...",2019-04-25,Mato Grosso,Centro-Oeste,valtenir pereira mdbmt regimentalidade legisla...,Não reeleito,ciência tecnologia


In [ ]:
selecao1['partido'].unique()

array(['DEM', 'PSD', 'SOLIDARI', 'PSL', 'UNIÃO', 'PSB', 'NOVO', 'PL',
       'PTB', 'MDB', 'CIDADANIA', 'PT', 'PSOL', 'PDT', 'PRB', 'REPUBLIC',
       'PR', 'PSC', 'PP', 'PCdoB', 'PROS', 'S.PART.', 'PSDB', 'PODE',
       'PV', 'PPS', 'PATRIOTA', 'PATRI', 'AVANTE', 'PRP', 'REDE'],
      dtype=object)

In [ ]:
temas_df = pd.DataFrame([['liberdade expressão', 'Liberdade de expressão'], ['liberdade imprensa', 'Liberdade de imprensa'], 
                         ['acesso informação', 'Lei de Acesso à Informação'], ['banco dados', 'Banco de dados públicos'], 
                         ['mercado câmbio', 'Marco legal para mercado de câmbio'], ['crédito consignado', 'Empréstimo consignado'], 
                         ['bolsa familia', 'Bolsa Família'], ['legal startups', 'Marco legal das Startups'], ['preço combustíveis', 'Preço dos combustíveis'], 
                         ['liberdade econômica', 'Liberdade econômica'], ['incentivo fiscal', 'Incentivo fiscal'], ['defesa empreendedor', 'Lei de Defeda do Empreendedor'], 
                         ['dados abertos', 'Dados Abertos'], ['auxílio emergencial', 'Auxílio Emergencial'], ['segurança pública', 'Segurança Pública'], 
                         ['meio ambiente', 'Meio ambiente'], ['seguridade social', 'Seguridade Social'], ['agricultura pecuária', 'Agricultura e pecuária'],
                         ['ciência tecnologia', 'Ciência e tecnologia'], ['direitos humanos', 'Direitos humanos'], ['reforma previdência', 'Reforma da Previdência'], 
                         ['reforma administrativa', 'Reforma administrativa'], ['relações exteriores', 'Relações exteriores'], ['defesa mulher', 'Defesa da mulher'], 
                         ['liberdade política', 'Liberdade política e econômica'], ['contribuição social', 'Arrecadação de impostos'], ['financeira orçamentária', 'Gestão orçamentária e financeira'], 
                         ['orçamentária financeira', 'Gestão orçamentária e financeira'], ['responsabilidade fiscal', 'Responsabilidade fiscal'], ['teto gasto', 'Teto de gastos'], 
                         ['teto gastos', 'Teto de gastos'], ['único saúde', 'SUS'], ['auxílio brasil', 'Auxílio Brasil'], ['povos indígenas', 'Proteção aos povos indígenas']],
                        columns=['ngrams', 'temas'])
temas_df

,ngrams,temas
0,liberdade expressão,Liberdade de expressão
1,liberdade imprensa,Liberdade de imprensa
2,acesso informação,Lei de Acesso à Informação
3,banco dados,Banco de dados públicos
4,mercado câmbio,Marco legal para mercado de câmbio
5,crédito consignado,Empréstimo consignado
6,bolsa familia,Bolsa Família
7,legal startups,Marco legal das Startups
8,preço combustíveis,Preço dos combustíveis
9,liberdade econômica,Liberdade econômica


In [ ]:
selecao_final = pd.merge(selecao1, temas_df, on='ngrams')
selecao_final

,id_proposicao,id_deputado,politico,partido,uf,ano,tipo,ementa,palavras_chave,status,data,estado,região,ementa_limpa,eleição,ngrams,temas
0,2222074,204457,juninho do pneu,DEM,RJ,2019,Emenda na Comissão,Altera o art. 195 para permitir a criação de c...,NaN,Relatório de Conferência de Assinaturas da EMC...,2019-09-26,Rio de Janeiro,Sudeste,permitir contribuição social rendimentos títul...,Reeleito(a),contribuição social,Arrecadação de impostos
1,2205885,204457,juninho do pneu,DEM,RJ,2019,Emenda na Comissão,Adicione-se ao texto da Proposta de Emenda à C...,NaN,Relatório de Conferência de Assinaturas da EMC...,2019-05-30,Rio de Janeiro,Sudeste,adicionese texto constituição seguintes iia es...,Reeleito(a),contribuição social,Arrecadação de impostos
2,2225870,141450,hugo leal,PSD,RJ,2019,Emenda na Comissão,Introduz vedação constitucional à dedução cont...,NaN,Relatório de Conferência de Assinaturas da EMC...,2019-10-17,Rio de Janeiro,Sudeste,introduz vedação constitucional dedução contáb...,Reeleito(a),contribuição social,Arrecadação de impostos
3,2222074,141450,hugo leal,PSD,RJ,2019,Emenda na Comissão,Altera o art. 195 para permitir a criação de c...,NaN,Relatório de Conferência de Assinaturas da EMC...,2019-09-26,Rio de Janeiro,Sudeste,permitir contribuição social rendimentos títul...,Reeleito(a),contribuição social,Arrecadação de impostos
4,2214938,141450,hugo leal,PSD,RJ,2019,Emenda na Comissão,Altera o artigo 159 da Constituição para acres...,NaN,Relatório de Conferência de Assinaturas da EMC...,2019-08-14,Rio de Janeiro,Sudeste,constituição acrescentar contribuição social p...,Reeleito(a),contribuição social,Arrecadação de impostos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15793,2283298,204536,kim kataguiri,DEM,SP,2021,Parecer do Relator,"Institui a Lei de Dados Abertos, estabelecendo...",NaN,"Apresentação do Parecer do Relator n. 2 CTASP,...",2021-05-18,São Paulo,Sudeste,dados abertos estabelecendo comitê gestor dado...,Reeleito(a),dados abertos,Dados Abertos
15794,2190511,204530,rodrigo agostinho,PSB,SP,2019,Projeto de Lei,Institui a Política Nacional de Dados Abertos ...,"Criação, Política Nacional de Dados Abertos, d...","Recebimento pela CCJC, apensado ao PL-11118/2018",2022-12-06,São Paulo,Sudeste,política dados abertos providências,Não reeleito,dados abertos,Dados Abertos
15795,2222471,204534,tabata amaral,PDT,SP,2019,Requerimento,Requer ao Senhor Ministro de Estado do Meio Am...,NaN,Aprovado,2019-10-01,São Paulo,Sudeste,estado meio ambiente ações monitoramento contr...,Reeleito(a),dados abertos,Dados Abertos
15796,2191656,162332,luiz nishimori,PR,PR,2019,Projeto de Lei,Institui a Política Nacional de Dados Abertos ...,"Criação, Política Nacional de Dados Abertos, d...","Recebimento pela CCJC, apensado ao PL-11118/2018",2022-12-06,Paraná,Sul,política dados abertos providências,Reeleito(a),dados abertos,Dados Abertos


In [ ]:
selecao_final['ano'].value_counts()

2019    6351
2021    4571
2020    2755
2022    2121
Name: ano, dtype: int64

In [ ]:
selecao_final.temas.value_counts()

Seguridade Social                     2576
Segurança Pública                     2170
Auxílio Emergencial                   1506
Meio ambiente                         1301
Arrecadação de impostos               1229
Gestão orçamentária e financeira      1223
Agricultura e pecuária                1025
SUS                                    870
Reforma da Previdência                 763
Relações exteriores                    654
Ciência e tecnologia                   518
Reforma administrativa                 442
Teto de gastos                         276
Proteção aos povos indígenas           242
Incentivo fiscal                       204
Auxílio Brasil                         166
Lei de Acesso à Informação             148
Defesa da mulher                       123
Responsabilidade fiscal                 92
Liberdade de expressão                  60
Empréstimo consignado                   41
Marco legal das Startups                33
Preço dos combustíveis                  25
Marco legal

In [ ]:
selecao_final.to_csv('/content/drive/MyDrive/Radar Virtù/Radar Congresso/Dados analisados/proposicoes_deputados_federais.csv', index=False)
# files.download('proposicoes_deputados_federais.csv')

In [ ]:
selecao_final[selecao_final['partido'].isnull()]
selecao_final.dropna(subset = ['partido'], inplace = True)
selecao_final.index = range(selecao_final.shape[0])
selecao_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15798 entries, 0 to 15797
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id_proposicao   15798 non-null  int64  
 1   id_deputado     15791 non-null  float64
 2   politico        15798 non-null  object 
 3   partido         15798 non-null  object 
 4   uf              15798 non-null  object 
 5   ano             15798 non-null  object 
 6   tipo            15798 non-null  object 
 7   ementa          15798 non-null  object 
 8   palavras_chave  2078 non-null   object 
 9   status          15796 non-null  object 
 10  data            15798 non-null  object 
 11  estado          15798 non-null  object 
 12  região          15798 non-null  object 
 13  ementa_limpa    15798 non-null  object 
 14  eleição         15798 non-null  object 
 15  ngrams          15798 non-null  object 
 16  temas           15798 non-null  object 
dtypes: float64(1), int64(1), object

In [ ]:
lista_dep_uf_partido = selecao_final[['politico', 'uf', 'partido']]
lista_dep_uf_partido.drop_duplicates()

,politico,uf,partido
0,juninho do pneu,RJ,DEM
2,hugo leal,RJ,PSD
12,aureo ribeiro,RJ,SOLIDARI
16,helio lopes,RJ,PSL
17,alessandro molon,RJ,PSB
...,...,...,...
14253,marcos pereira,SP,REPUBLIC
14360,alex manente,SP,PPS
14440,soraya thronicke,MS,PSL
15755,raimundo costa,BA,PODE


In [ ]:
lista_dep_uf_partido.to_csv('/content/drive/MyDrive/Radar Virtù/Radar Congresso/Dados analisados/lista_uf_partido.csv', index=False)